# 🤖 Modelado Supervisado H2O + GPU + Paralelo - Precios Inmobiliarios Dinamarca

**Objetivo**: Construir modelos distribuidos H2O, clásicos sklearn y de árbol con optimizaciones GPU y paralelización, evaluarlos con métricas sólidas considerando rendimiento y velocidad.

1. División de Datos (Temporal)

    1.1 Split temporal (80% train, 20% test)

    1.2 Validación de distribución en train/test

    1.3 Visualización de drift o cambios en el tiempo (si aplica)

2. Modelos Estadísticos

    2.1 Regresión Lineal, Ridge, Lasso, ElasticNet

    2.2 Ajuste con mínimos cuadrados o likelihood

    2.3 Comparación de modelos con AIC/BIC (sin CV)

    2.4 Diagnóstico de residuos y detección de outliers

3. Modelos de Árboles

    3.1 LightGBM y Random Forest (hiperparámetros por defecto o mínimos)

    3.2 Comparación con modelos estadísticos

    3.3 Feature importance y explicabilidad inicial

6. Evaluación General

    6.1 Métricas: RMSE, MAE, MAPE, R²

    6.2 Visualización de errores (residuos, pred vs real)

    6.3 Análisis por segmentos (tipo de casa, región)

    6.4 Tabla resumen de modelos y conclusiones

In [1]:
from setup import set_project_root
set_project_root()

from config import *
from descriptive_analysis import (
    load_and_validate_data,
)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings




Current working directory: /home/exodia/Documentos/TFBigData


In [2]:
df_clean, h2o_frame = load_and_validate_data(
    data_path=CLEAN_FILE, 
    destination_frame=DESTINATION_FRAME, 
    parallel=False
)




Checking whether there is an H2O instance running at http://localhost:54321......... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.8" 2023-07-18; OpenJDK Runtime Environment Temurin-17.0.8+7 (build 17.0.8+7); OpenJDK 64-Bit Server VM Temurin-17.0.8+7 (build 17.0.8+7, mixed mode, sharing)
  Starting server from /home/exodia/miniconda3/envs/TFBigData/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpugqe8252
  JVM stdout: /tmp/tmpugqe8252/h2o_exodia_started_from_python.out
  JVM stderr: /tmp/tmpugqe8252/h2o_exodia_started_from_python.err
 not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.8" 2023-07-18; OpenJDK Runtime Environment Temurin-17.0.8+7 (build 17.0.8+7); OpenJDK 64-Bit Server VM Temurin-17.0.8+7 (build 17.0.8+7, mixed mode, sharing)
  Starting server from /home/exodia/miniconda3/envs/TFBigData/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpugqe8

H2O_cluster_uptime:,00 secs
H2O_cluster_timezone:,America/Lima
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,3 months and 15 days
H2O_cluster_name:,H2O_from_python_exodia_7tsk3w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.723 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Importando datos desde /home/exodia/Documentos/TFBigData/data/processed/cleaned_data.parquet

Parse progress: |Parse progress: |██████████████████████████████████████████████████████████████████████| (done) 100%
██████████████████████████████████████████████████████████| (done) 100%


KeyboardInterrupt: 

In [ ]:

PROCESSED_DIR = DATA_DIR / "processed"

print(f"📂 Directorio procesados: {PROCESSED_DIR}")

# Intentar cargar datos procesados con múltiples formatos
df_processed = None
processed_files = [
    PROCESSED_DIR / "cleaned_data.parquet",
    PROCESSED_DIR / "processed_data.parquet", 
    PROCESSED_DIR / "feature_engineered_data.parquet",
    PROCESSED_DIR / "final_data.parquet",
    PROCESSED_DIR / "cleaned_data.csv"
]

for file_path in processed_files:
    if file_path.exists():
        try:
            print(f"🔄 Intentando cargar: {file_path.name}")
            
            if file_path.suffix == '.parquet':
                df_processed = pd.read_parquet(file_path)
            elif file_path.suffix == '.csv':
                df_processed = pd.read_csv(file_path)
            
            print(f"✅ Datos procesados cargados desde: {file_path.name}")
            print(f"   Dimensiones: {df_processed.shape[0]:,} filas x {df_processed.shape[1]} columnas")
            break
            
        except Exception as e:
            print(f"❌ Error cargando {file_path.name}: {e}")
            continue

# Si no se encuentran datos procesados, usar datos limpios como fallback
if df_processed is None:
    print("⚠️ No se encontraron datos procesados, usando datos limpios como fallback")
    df_processed = df_clean.copy()
    print(f"   Usando datos limpios: {df_processed.shape[0]:,} filas x {df_processed.shape[1]} columnas")

# Verificar estructura de datos procesados
print(f"\n🔍 Verificando estructura de datos procesados:")
print(f"   Columnas totales: {len(df_processed.columns)}")
print(f"   Tipos de datos: {df_processed.dtypes.value_counts().to_dict()}")

# Verificar si el target está presente
if TARGET in df_processed.columns:
    print(f"✅ Variable objetivo '{TARGET}' encontrada")
    print(f"   Estadísticas del target:")
    print(f"     Media: {df_processed[TARGET].mean():.2f}")
    print(f"     Mediana: {df_processed[TARGET].median():.2f}")
    print(f"     Std: {df_processed[TARGET].std():.2f}")
    print(f"     Valores nulos: {df_processed[TARGET].isnull().sum()}")
else:
    print(f"❌ Variable objetivo '{TARGET}' NO encontrada")
    print(f"   Columnas disponibles: {list(df_processed.columns)}")

# Verificar calidad de datos
print(f"\n📊 Calidad de datos procesados:")
total_nulls = df_processed.isnull().sum().sum()
print(f"   Valores nulos totales: {total_nulls:,}")

if total_nulls > 0:
    null_cols = df_processed.isnull().sum()
    null_cols = null_cols[null_cols > 0].sort_values(ascending=False)
    print(f"   Columnas con nulls: {len(null_cols)}")
    for col, nulls in null_cols.head(10).items():
        print(f"     {col}: {nulls:,} nulls ({nulls/len(df_processed)*100:.1f}%)")

# Identificar tipos de columnas para el procesamiento
categorical_cols = df_processed.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()

print(f"\n🏷️ Tipos de variables identificadas:")
print(f"   Categóricas: {len(categorical_cols)}")
print(f"   Numéricas: {len(numerical_cols)}")

# Preparar lista de features excluyendo columnas problemáticas
features_to_exclude = [TARGET] + DROP_COLS
if 'year' in df_processed.columns:
    features_to_exclude.append('year')
if 'Year' in df_processed.columns:
    features_to_exclude.append('Year')

# Lista final de features disponibles
available_features = [col for col in df_processed.columns 
                     if col not in features_to_exclude]

print(f"\n🎯 Features para modelado:")
print(f"   Total disponibles: {len(available_features)}")
print(f"   Excluidas: {features_to_exclude}")

# Mostrar muestra de los datos procesados
print(f"\n📋 Muestra de datos procesados:")
display(df_processed.head(3))

print(f"\n✅ Carga de datos procesados completada")
print(f"🚀 Listo para conversión multiframework")

📁 Cargando datos procesados desde Feature Engineering (Notebook 03)...
📂 Directorio procesados: /home/exodia/Documentos/TFBigData/data/processed
🔄 Intentando cargar: cleaned_data.parquet


✅ Datos procesados cargados desde: cleaned_data.parquet
   Dimensiones: 1,506,591 filas x 19 columnas

🔍 Verificando estructura de datos procesados:
   Columnas totales: 19
   Tipos de datos: {dtype('int64'): 8, dtype('O'): 8, dtype('float64'): 3}
✅ Variable objetivo 'purchase_price' encontrada
   Estadísticas del target:
     Media: 1915290.96
     Mediana: 1400000.00
     Std: 1765457.84
     Valores nulos: 0

📊 Calidad de datos procesados:
   Valores nulos totales: 0

🏷️ Tipos de variables identificadas:
   Categóricas: 8
   Numéricas: 11

🎯 Features para modelado:
   Total disponibles: 14
   Excluidas: ['purchase_price', 'house_id', 'address', 'sqm_price', '%_change_between_offer_and_purchase']

📋 Muestra de datos procesados:
   Valores nulos totales: 0

🏷️ Tipos de variables identificadas:
   Categóricas: 8
   Numéricas: 11

🎯 Features para modelado:
   Total disponibles: 14
   Excluidas: ['purchase_price', 'house_id', 'address', 'sqm_price', '%_change_between_offer_and_purchase']

,date,quarter,house_id,house_type,sales_type,year_build,purchase_price,%_change_between_offer_and_purchase,no_rooms,sqm,sqm_price,address,zip_code,city,area,region,nom_interest_rate%,dk_ann_infl_rate%,yield_on_mortgage_credit_bonds%
0,1727654400000000000,218,1300,Apartment,regular_sale,1971,1765000,0,3,78.0,22628.205078,"Hedekæret 38, 1. th",2640,Hedehusene,"Capital, Copenhagen",Zealand,3.35,1.1299999952316284,4.340000152587891
1,1727654400000000000,218,1307,Summerhouse,regular_sale,2009,590939,0,3,50.0,11818.780273,Violstien 11,2635,Ishøj,"Capital, Copenhagen",Zealand,3.35,1.1299999952316284,4.340000152587891
2,1727654400000000000,218,1301,Apartment,regular_sale,1940,1750000,0,2,56.0,31250.000000,"Buddingevej 72I, st. tv",2800,Kongens Lyngby,"Capital, Copenhagen",Zealand,3.35,1.1299999952316284,4.340000152587891



✅ Carga de datos procesados completada
🚀 Listo para conversión multiframework


In [ ]:
# ==================================================
# VERIFICACIÓN DE CONFIGURACIÓN OPTIMIZADA
# ==================================================

print("🔧 Verificando configuración GPU y paralelización...")

# Verificar configuración de GPU
def check_gpu_setup():
    """Verifica disponibilidad y configuración de GPU"""
    gpu_info = {
        'cuda_available': False,
        'gpu_count': 0,
        'gpu_names': [],
        'lightgbm_gpu': False
    }
    
    # Verificar CUDA/GPU
    try:
        import torch
        if torch.cuda.is_available():
            gpu_info['cuda_available'] = True
            gpu_info['gpu_count'] = torch.cuda.device_count()
            gpu_info['gpu_names'] = [torch.cuda.get_device_name(i) for i in range(gpu_info['gpu_count'])]
        print(f"🎮 CUDA disponible: {gpu_info['cuda_available']}")
        if gpu_info['cuda_available']:
            print(f"   GPUs detectadas: {gpu_info['gpu_count']}")
            for i, name in enumerate(gpu_info['gpu_names']):
                print(f"   GPU {i}: {name}")
    except ImportError:
        print("⚠️ PyTorch no instalado - verificación CUDA limitada")
    
    # Verificar LightGBM GPU
    try:
        test_model = lgb.LGBMRegressor(device='gpu', gpu_platform_id=0, gpu_device_id=0)
        gpu_info['lightgbm_gpu'] = True
        print("✅ LightGBM GPU: Disponible")
    except Exception as e:
        print(f"⚠️ LightGBM GPU: No disponible - {str(e)[:100]}")
    
    return gpu_info

# Verificar configuración de paralelización
def check_parallel_setup():
    """Verifica configuración de paralelización"""
    import psutil
    
    parallel_info = {
        'cpu_count': mp.cpu_count(),
        'physical_cores': psutil.cpu_count(logical=False),
        'logical_cores': psutil.cpu_count(logical=True),
        'memory_gb': round(psutil.virtual_memory().total / (1024**3), 1),
        'available_memory_gb': round(psutil.virtual_memory().available / (1024**3), 1)
    }
    
    print(f"💻 CONFIGURACIÓN CPU:")
    print(f"   Cores físicos: {parallel_info['physical_cores']}")
    print(f"   Cores lógicos: {parallel_info['logical_cores']}")
    print(f"   Memoria total: {parallel_info['memory_gb']} GB")
    print(f"   Memoria disponible: {parallel_info['available_memory_gb']} GB")
    
    # Verificar configuración de librerías paralelas
    print(f"🔧 CONFIGURACIÓN PARALELA:")
    print(f"   OMP_NUM_THREADS: {os.environ.get('OMP_NUM_THREADS', 'No configurado')}")
    print(f"   MKL_NUM_THREADS: {os.environ.get('MKL_NUM_THREADS', 'No configurado')}")
    
    return parallel_info

# Ejecutar verificaciones
gpu_config = check_gpu_setup()
parallel_config = check_parallel_setup()

# Configurar estrategia de entrenamiento basada en recursos disponibles
TRAINING_STRATEGY = {
    'use_gpu': gpu_config['lightgbm_gpu'],
    'max_cores': parallel_config['logical_cores'],
    'memory_limit': parallel_config['available_memory_gb'],
    'batch_processing': parallel_config['available_memory_gb'] < 8  # Si memoria < 8GB, usar batches
}

print(f"\n📋 ESTRATEGIA DE ENTRENAMIENTO:")
print(f"   GPU para LightGBM: {'✅ Sí' if TRAINING_STRATEGY['use_gpu'] else '❌ No'}")
print(f"   Cores para paralelización: {TRAINING_STRATEGY['max_cores']}")
print(f"   Límite de memoria: {TRAINING_STRATEGY['memory_limit']} GB")
print(f"   Procesamiento en batches: {'✅ Sí' if TRAINING_STRATEGY['batch_processing'] else '❌ No'}")

# Optimizar configuración de numpy/scipy para rendimiento
import numpy as np
if hasattr(np, '__config__'):
    print(f"\n⚙️ CONFIGURACIÓN NUMPY:")
    try:
        print(f"   BLAS: {np.__config__.get_info('blas_info', {}).get('name', 'Desconocido')}")
        print(f"   LAPACK: {np.__config__.get_info('lapack_info', {}).get('name', 'Desconocido')}")
    except:
        print("   Información BLAS/LAPACK no disponible")

print(f"\n✅ Verificación de configuración completada")
print(f"🚀 Sistema optimizado para entrenamiento de alta velocidad")

🔧 Verificando configuración GPU y paralelización...
⚠️ PyTorch no instalado - verificación CUDA limitada
✅ LightGBM GPU: Disponible
💻 CONFIGURACIÓN CPU:
   Cores físicos: 16
   Cores lógicos: 16
   Memoria total: 26.9 GB
   Memoria disponible: 5.3 GB
🔧 CONFIGURACIÓN PARALELA:
   OMP_NUM_THREADS: 16
   MKL_NUM_THREADS: 16

📋 ESTRATEGIA DE ENTRENAMIENTO:
   GPU para LightGBM: ✅ Sí
   Cores para paralelización: 16
   Límite de memoria: 5.3 GB
   Procesamiento en batches: ✅ Sí

⚙️ CONFIGURACIÓN NUMPY:
   Información BLAS/LAPACK no disponible

✅ Verificación de configuración completada
🚀 Sistema optimizado para entrenamiento de alta velocidad


## 1. División de Datos (Temporal)

### Estrategia de División Temporal
- **Train**: 80% de los datos más antiguos (1992-2019)
- **Test**: 20% de los datos más recientes (2020-2024)
- **Validación**: Sin data leakage temporal

In [ ]:
# ==================================================
# 1.1 DIVISIÓN TEMPORAL H2O + SKLEARN (80% TRAIN, 20% TEST)
# ==================================================

print("📅 Implementando división temporal optimizada para H2O...")

# Verificar si tenemos columna temporal
if 'year' in df_clean.columns:
    # Información temporal
    year_min, year_max = df_clean['year'].min(), df_clean['year'].max()
    print(f"📊 Rango temporal completo: {year_min} - {year_max}")
    
    # División temporal: 80% para train, 20% para test
    years_unique = sorted(df_clean['year'].unique())
    n_years = len(years_unique)
    split_index = int(n_years * 0.8)
    split_year = years_unique[split_index]
    
    print(f"🔄 División temporal:")
    print(f"  Train: {year_min} - {split_year-1} ({split_index} años)")
    print(f"  Test:  {split_year} - {year_max} ({n_years - split_index} años)")
    
    # Crear máscaras para train/test
    train_mask = df_clean['year'] < split_year
    test_mask = df_clean['year'] >= split_year
    
    # División para H2O
    if H2O_AVAILABLE and h2o_frame is not None:
        print("🚀 Creando splits en H2O...")
        
        # Dividir H2O Frame
        h2o_train = h2o_frame[h2o_frame['year'] < split_year, :]
        h2o_test = h2o_frame[h2o_frame['year'] >= split_year, :]
        
        # Definir predictores (excluir target y columnas auxiliares)
        h2o_predictors = [col for col in h2o_frame.columns 
                         if col not in [TARGET, 'year'] + DROP_COLS]
        
        print(f"✅ H2O splits creados:")
        print(f"  Train: {h2o_train.nrows:,} filas")
        print(f"  Test:  {h2o_test.nrows:,} filas")
        print(f"  Predictores: {len(h2o_predictors)} variables")
    
    # División para sklearn (backup/comparación)
    print("📊 Creando splits para sklearn...")
    
    # Filtrar features disponibles para sklearn
    available_features_sklearn = [f for f in available_features if f in df_clean.columns]
    
    # Crear conjuntos de datos para sklearn
    X_train = df_clean[train_mask][available_features_sklearn]
    X_test = df_clean[test_mask][available_features_sklearn]
    y_train = df_clean[train_mask][TARGET]
    y_test = df_clean[test_mask][TARGET]
    
    # Información de la división
    train_size = len(X_train)
    test_size = len(X_test)
    total_size = train_size + test_size
    
    print(f"✅ Sklearn splits creados:")
    print(f"  Train: {train_size:,} observaciones ({train_size/total_size*100:.1f}%)")
    print(f"  Test:  {test_size:,} observaciones ({test_size/total_size*100:.1f}%)")
    print(f"  Features: {len(available_features_sklearn)} variables")
    
    # Verificar distribución temporal
    train_years = df_clean[train_mask]['year'].value_counts().sort_index()
    test_years = df_clean[test_mask]['year'].value_counts().sort_index()
    
    print(f"\n📊 Distribución temporal verificada:")
    print(f"  Train años: {train_years.index.min()} - {train_years.index.max()}")
    print(f"  Test años:  {test_years.index.min()} - {test_years.index.max()}")
    
    # Guardar información de split
    split_info = {
        'split_year': split_year,
        'train_years': (train_years.index.min(), train_years.index.max()),
        'test_years': (test_years.index.min(), test_years.index.max()),
        'train_size': train_size,
        'test_size': test_size,
        'h2o_available': H2O_AVAILABLE,
        'h2o_predictors': h2o_predictors if H2O_AVAILABLE else None
    }
    
else:
    print("❌ Error: No se encontró la columna 'year' en el dataset")
    print("   Se requiere información temporal para división correcta")

print(f"\n✅ División temporal completada exitosamente")

📅 Implementando división temporal optimizada para H2O...
❌ Error: No se encontró la columna 'year' en el dataset
   Se requiere información temporal para división correcta

✅ División temporal completada exitosamente


In [ ]:
# DIVISIÓN TEMPORAL DE DATOS PROCESADOS

print("⏰ Ejecutando división temporal con datos procesados del notebook 03...")

# Verificar que tenemos los datos procesados
if 'df_processed' not in locals():
    raise ValueError("❌ df_processed no está definido. Ejecutar primero la celda de carga de datos.")

if TARGET not in df_processed.columns:
    raise ValueError(f"❌ Variable objetivo '{TARGET}' no encontrada en datos procesados")

print(f"✅ Datos procesados disponibles: {df_processed.shape}")
print(f"   Target: {TARGET}")
print(f"   Features disponibles: {len(available_features)}")

# Preparar datos para división
X = df_processed[available_features].copy()
y = df_processed[TARGET].copy()

# Limpiar datos (remover nulls en target)
mask_valid = y.notna()
X = X[mask_valid]
y = y[mask_valid]

print(f"📊 Datos limpios para modelado:")
print(f"   Muestras válidas: {len(X):,}")
print(f"   Features: {len(available_features)}")

# División temporal si existe columna de fecha
from sklearn.model_selection import train_test_split

# Buscar columna temporal
date_columns = ['date', 'Date', 'fecha', 'timestamp', 'SaleDate', 'SaleDato']
temporal_col = None

for col in date_columns:
    if col in df_processed.columns:
        temporal_col = col
        break

# División temporal o aleatoria
if temporal_col is not None:
    print(f"🕐 Usando división temporal con columna: {temporal_col}")
    
    # Preparar datos con fechas válidas
    df_temp = df_processed[mask_valid].copy()
    
    try:
        # Convertir a datetime si es necesario
        if df_temp[temporal_col].dtype == 'object':
            df_temp[temporal_col] = pd.to_datetime(df_temp[temporal_col], errors='coerce')
        
        # Filtrar fechas válidas
        date_mask = df_temp[temporal_col].notna()
        df_temp = df_temp[date_mask]
        
        if len(df_temp) > 0:
            # Ordenar por fecha y dividir temporalmente
            df_temp = df_temp.sort_values(temporal_col)
            split_point = int(len(df_temp) * 0.8)
            
            train_indices = df_temp.index[:split_point]
            test_indices = df_temp.index[split_point:]
            
            X_train = X.loc[train_indices]
            X_test = X.loc[test_indices]
            y_train = y.loc[train_indices]
            y_test = y.loc[test_indices]
            
            print(f"   ✅ División temporal exitosa")
            print(f"   Período train: {df_temp[temporal_col].iloc[0]} a {df_temp[temporal_col].iloc[split_point-1]}")
            print(f"   Período test: {df_temp[temporal_col].iloc[split_point]} a {df_temp[temporal_col].iloc[-1]}")
        else:
            raise ValueError("No hay fechas válidas")
            
    except Exception as e:
        print(f"   ⚠️ Error en división temporal: {e}")
        print(f"   Usando división aleatoria como fallback")
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
else:
    print(f"📊 Usando división aleatoria (no se encontró columna temporal)")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

# Verificar división
print(f"\n📈 Resultados de la división:")
print(f"   Train: {len(X_train):,} muestras ({len(X_train)/len(X)*100:.1f}%)")
print(f"   Test: {len(X_test):,} muestras ({len(X_test)/len(X)*100:.1f}%)")

# Estadísticas del target
print(f"\n🎯 Estadísticas del target '{TARGET}':")
print(f"   Train - Media: {y_train.mean():.2f}, Std: {y_train.std():.2f}")
print(f"   Test - Media: {y_test.mean():.2f}, Std: {y_test.std():.2f}")
print(f"   Diferencia medias: {abs(y_train.mean() - y_test.mean()):.2f}")

# Verificar nulls en features
train_nulls = X_train.isnull().sum().sum()
test_nulls = X_test.isnull().sum().sum()

if train_nulls > 0 or test_nulls > 0:
    print(f"⚠️ Nulls encontrados - Train: {train_nulls}, Test: {test_nulls}")
    
    # Imputación simple si hay nulls
    from sklearn.impute import SimpleImputer
    
    # Separar columnas por tipo
    numeric_features = X_train.select_dtypes(include=[np.number]).columns
    categorical_features = X_train.select_dtypes(exclude=[np.number]).columns
    
    # Imputar numéricas con mediana
    if len(numeric_features) > 0:
        imputer_num = SimpleImputer(strategy='median')
        X_train[numeric_features] = imputer_num.fit_transform(X_train[numeric_features])
        X_test[numeric_features] = imputer_num.transform(X_test[numeric_features])
    
    # Imputar categóricas con moda
    if len(categorical_features) > 0:
        imputer_cat = SimpleImputer(strategy='most_frequent')
        X_train[categorical_features] = imputer_cat.fit_transform(X_train[categorical_features])
        X_test[categorical_features] = imputer_cat.transform(X_test[categorical_features])
    
    print(f"✅ Imputación completada")

print(f"\n✅ División de datos completada")
print(f"🚀 Variables train/test listas para modelado")

⏰ Ejecutando división temporal con datos procesados del notebook 03...
✅ Datos procesados disponibles: (1506591, 19)
   Target: purchase_price
   Features disponibles: 14
📊 Datos limpios para modelado:
   Muestras válidas: 1,506,591
   Features: 14
🕐 Usando división temporal con columna: date
   ✅ División temporal exitosa
   Período train: 694569600000000000 a 1619222400000000000
   Período test: 1619222400000000000 a 1727654400000000000

📈 Resultados de la división:
   Train: 1,205,272 muestras (80.0%)
   Test: 301,319 muestras (20.0%)

🎯 Estadísticas del target 'purchase_price':
   Train - Media: 1762487.06, Std: 1651134.06
   Test - Media: 2526504.54, Std: 2052379.56
   Diferencia medias: 764017.48

✅ División de datos completada
🚀 Variables train/test listas para modelado
   ✅ División temporal exitosa
   Período train: 694569600000000000 a 1619222400000000000
   Período test: 1619222400000000000 a 1727654400000000000

📈 Resultados de la división:
   Train: 1,205,272 muestras (80.

In [ ]:
# ==================================================
# 1.3 VISUALIZACIÓN DE DRIFT TEMPORAL
# ==================================================

print("📈 Analizando drift temporal en variables clave...")

def analyze_temporal_drift(df_complete, features_to_analyze, target_col):
    """Analiza drift temporal en variables clave"""
    
    # Seleccionar subset de features importantes para análisis
    important_features = features_to_analyze[:6]  # Top 6 features
    
    n_features = len(important_features)
    n_cols = 3
    n_rows = (n_features + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 4*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
    
    # Análisis por año
    yearly_stats = df_complete.groupby('year').agg({
        target_col: ['mean', 'std', 'count'],
        **{feature: 'mean' for feature in important_features}
    }).reset_index()
    
    # Evolución de la variable objetivo
    for i, feature in enumerate(important_features):
        if i < len(axes):
            # Plot de evolución temporal
            if feature in df_complete.columns:
                feature_yearly = df_complete.groupby('year')[feature].mean()
                axes[i].plot(feature_yearly.index, feature_yearly.values, marker='o', linewidth=2)
                axes[i].set_title(f'Evolución Temporal: {feature}')
                axes[i].set_xlabel('Año')
                axes[i].set_ylabel(f'{feature}')
                axes[i].grid(True, alpha=0.3)
                
                # Marcar división train/test
                axes[i].axvline(x=split_year, color='red', linestyle='--', alpha=0.7, label='Train/Test Split')
                axes[i].legend()
    
    # Ocultar ejes no utilizados
    for i in range(n_features, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Análisis específico del precio por año
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Precio promedio por año
    price_yearly = df_complete.groupby('year')[target_col].agg(['mean', 'std', 'count'])
    
    axes[0,0].plot(price_yearly.index, price_yearly['mean'], marker='o', linewidth=2, color='blue')
    axes[0,0].fill_between(price_yearly.index, 
                          price_yearly['mean'] - price_yearly['std'],
                          price_yearly['mean'] + price_yearly['std'], 
                          alpha=0.3, color='blue')
    axes[0,0].axvline(x=split_year, color='red', linestyle='--', alpha=0.7, label='Train/Test Split')
    axes[0,0].set_title('Evolución del Precio Promedio ± 1 Std')
    axes[0,0].set_xlabel('Año')
    axes[0,0].set_ylabel('Precio (DKK)')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # Número de transacciones por año
    axes[0,1].bar(price_yearly.index, price_yearly['count'], color='green', alpha=0.7)
    axes[0,1].axvline(x=split_year, color='red', linestyle='--', alpha=0.7, label='Train/Test Split')
    axes[0,1].set_title('Número de Transacciones por Año')
    axes[0,1].set_xlabel('Año')
    axes[0,1].set_ylabel('Número de Transacciones')
    axes[0,1].legend()
    
    # Coeficiente de variación por año
    cv_yearly = (price_yearly['std'] / price_yearly['mean']) * 100
    axes[1,0].plot(cv_yearly.index, cv_yearly.values, marker='s', linewidth=2, color='orange')
    axes[1,0].axvline(x=split_year, color='red', linestyle='--', alpha=0.7, label='Train/Test Split')
    axes[1,0].set_title('Coeficiente de Variación del Precio (%)')
    axes[1,0].set_xlabel('Año')
    axes[1,0].set_ylabel('CV (%)')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # Distribución de precios por período
    train_data = df_complete[df_complete['year'] < split_year][target_col]
    test_data = df_complete[df_complete['year'] >= split_year][target_col]
    
    axes[1,1].hist(train_data, bins=50, alpha=0.7, label='Train Period', density=True, color='blue')
    axes[1,1].hist(test_data, bins=50, alpha=0.7, label='Test Period', density=True, color='red')
    axes[1,1].set_title('Distribución de Precios: Train vs Test')
    axes[1,1].set_xlabel('Precio (DKK)')
    axes[1,1].set_ylabel('Densidad')
    axes[1,1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Análisis cuantitativo del drift
    print(f"\n📊 Análisis cuantitativo de drift temporal:")
    print(f"{'Variable':<20} {'Drift Train->Test':<20} {'Significancia':<15}")
    print("-" * 60)
    
    for feature in important_features[:5]:  # Top 5 features
        if feature in df_complete.columns:
            train_mean = df_complete[df_complete['year'] < split_year][feature].mean()
            test_mean = df_complete[df_complete['year'] >= split_year][feature].mean()
            drift_pct = ((test_mean - train_mean) / train_mean) * 100 if train_mean != 0 else 0
            
            # Test de significancia
            train_values = df_complete[df_complete['year'] < split_year][feature]
            test_values = df_complete[df_complete['year'] >= split_year][feature]
            _, p_value = stats.ttest_ind(train_values, test_values)
            
            significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
            
            print(f"{feature:<20} {drift_pct:<20.2f}% {significance:<15}")

# Ejecutar análisis de drift
analyze_temporal_drift(df_complete, available_features, TARGET)
analyze_temporal_drift(df_complete, available_features, TARGET)

## 3. Modelos Estadísticos (Sklearn)

### Metodología Estadística Clásica de Comparación
- **Regresión Lineal**: Baseline con mínimos cuadrados
- **Ridge**: Regularización L2 para multicolinealidad
- **Lasso**: Regularización L1 para selección de features
- **ElasticNet**: Combinación L1 + L2
- **Evaluación**: AIC/BIC + GridSearch paralelo para comparación con H2O

## 2. Modelos H2O (Distribuidos y Escalables)

### Metodología H2O AutoML y Modelos Específicos
- **H2O GBM**: Gradient Boosting Machine distribuido y optimizado
- **H2O Random Forest**: Ensemble distribuido con paralelización automática
- **H2O GLM**: Modelos lineales generalizados con regularización automática
- **H2O Deep Learning**: Redes neuronales profundas para capturar relaciones complejas
- **H2O AutoML**: Pipeline automatizado para selección del mejor modelo
- **Evaluación**: Métricas nativas H2O con validación cruzada optimizada

In [ ]:
# ==================================================
# 2.1 MODELOS H2O DISTRIBUIDOS Y ESCALABLES
# ==================================================

if H2O_AVAILABLE:
    print("🚀 Entrenando modelos H2O distribuidos...")
    
    # Diccionario para almacenar modelos H2O
    h2o_models = {}
    h2o_results = {}
    
    # 1. H2O Gradient Boosting Machine (GBM)
    print("\n🔸 Entrenando H2O GBM (Distribuido)...")
    start_time = datetime.now()
    
    h2o_gbm = H2OGradientBoostingEstimator(
        ntrees=100,
        max_depth=6,
        learn_rate=0.1,
        sample_rate=0.8,
        col_sample_rate=0.8,
        stopping_rounds=10,
        stopping_tolerance=0.001,
        stopping_metric="RMSE",
        seed=42,
        nfolds=5,  # Validación cruzada
        fold_assignment="Modulo",
        keep_cross_validation_predictions=True
    )
    
    h2o_gbm.train(
        x=h2o_predictors,
        y=TARGET,
        training_frame=h2o_train,
        validation_frame=h2o_test
    )
    
    gbm_time = (datetime.now() - start_time).total_seconds()
    h2o_models['H2O_GBM'] = h2o_gbm
    
    # Métricas H2O GBM
    train_rmse_gbm = h2o_gbm.rmse(train=True)
    valid_rmse_gbm = h2o_gbm.rmse(valid=True)
    train_r2_gbm = h2o_gbm.r2(train=True)
    valid_r2_gbm = h2o_gbm.r2(valid=True)
    
    h2o_results['H2O_GBM'] = {
        'train_rmse': train_rmse_gbm,
        'test_rmse': valid_rmse_gbm,
        'train_r2': train_r2_gbm,
        'test_r2': valid_r2_gbm,
        'training_time': gbm_time,
        'cv_rmse': h2o_gbm.rmse(xval=True),
        'cv_r2': h2o_gbm.r2(xval=True)
    }
    
    print(f"   Tiempo entrenamiento: {gbm_time:.2f}s")
    print(f"   Train RMSE: {train_rmse_gbm:,.0f}")
    print(f"   Test RMSE:  {valid_rmse_gbm:,.0f}")
    print(f"   Test R²:    {valid_r2_gbm:.4f}")
    print(f"   CV RMSE:    {h2o_gbm.rmse(xval=True):,.0f}")
    
    # 2. H2O Random Forest
    print("\n🔸 Entrenando H2O Random Forest (Distribuido)...")
    start_time = datetime.now()
    
    h2o_rf = H2ORandomForestEstimator(
        ntrees=100,
        max_depth=15,
        sample_rate=0.8,
        col_sample_rate_per_tree=0.8,
        seed=42,
        nfolds=5,
        fold_assignment="Modulo",
        keep_cross_validation_predictions=True
    )
    
    h2o_rf.train(
        x=h2o_predictors,
        y=TARGET,
        training_frame=h2o_train,
        validation_frame=h2o_test
    )
    
    rf_time = (datetime.now() - start_time).total_seconds()
    h2o_models['H2O_RF'] = h2o_rf
    
    # Métricas H2O RF
    train_rmse_rf = h2o_rf.rmse(train=True)
    valid_rmse_rf = h2o_rf.rmse(valid=True)
    train_r2_rf = h2o_rf.r2(train=True)
    valid_r2_rf = h2o_rf.r2(valid=True)
    
    h2o_results['H2O_RF'] = {
        'train_rmse': train_rmse_rf,
        'test_rmse': valid_rmse_rf,
        'train_r2': train_r2_rf,
        'test_r2': valid_r2_rf,
        'training_time': rf_time,
        'cv_rmse': h2o_rf.rmse(xval=True),
        'cv_r2': h2o_rf.r2(xval=True)
    }
    
    print(f"   Tiempo entrenamiento: {rf_time:.2f}s")
    print(f"   Train RMSE: {train_rmse_rf:,.0f}")
    print(f"   Test RMSE:  {valid_rmse_rf:,.0f}")
    print(f"   Test R²:    {valid_r2_rf:.4f}")
    print(f"   CV RMSE:    {h2o_rf.rmse(xval=True):,.0f}")
    
    # 3. H2O GLM (Generalized Linear Model)
    print("\n🔸 Entrenando H2O GLM (Regularizado)...")
    start_time = datetime.now()
    
    h2o_glm = H2OGeneralizedLinearEstimator(
        family="gaussian",
        alpha=0.5,  # Elastic Net
        lambda_search=True,
        nfolds=5,
        fold_assignment="Modulo",
        keep_cross_validation_predictions=True,
        seed=42
    )
    
    h2o_glm.train(
        x=h2o_predictors,
        y=TARGET,
        training_frame=h2o_train,
        validation_frame=h2o_test
    )
    
    glm_time = (datetime.now() - start_time).total_seconds()
    h2o_models['H2O_GLM'] = h2o_glm
    
    # Métricas H2O GLM
    train_rmse_glm = h2o_glm.rmse(train=True)
    valid_rmse_glm = h2o_glm.rmse(valid=True)
    train_r2_glm = h2o_glm.r2(train=True)
    valid_r2_glm = h2o_glm.r2(valid=True)
    
    h2o_results['H2O_GLM'] = {
        'train_rmse': train_rmse_glm,
        'test_rmse': valid_rmse_glm,
        'train_r2': train_r2_glm,
        'test_r2': valid_r2_glm,
        'training_time': glm_time,
        'cv_rmse': h2o_glm.rmse(xval=True),
        'cv_r2': h2o_glm.r2(xval=True),
        'best_lambda': h2o_glm.lambda_best()
    }
    
    print(f"   Tiempo entrenamiento: {glm_time:.2f}s")
    print(f"   Best lambda: {h2o_glm.lambda_best():.6f}")
    print(f"   Train RMSE: {train_rmse_glm:,.0f}")
    print(f"   Test RMSE:  {valid_rmse_glm::.0f}")
    print(f"   Test R²:    {valid_r2_glm:.4f}")
    print(f"   CV RMSE:    {h2o_glm.rmse(xval=True):,.0f}")
    
    # 4. H2O Deep Learning
    print("\n🔸 Entrenando H2O Deep Learning (Neural Network)...")
    start_time = datetime.now()
    
    h2o_dl = H2ODeepLearningEstimator(
        hidden=[200, 200, 100],  # 3 capas ocultas
        epochs=50,
        activation="Rectifier",
        l1=0.001,
        l2=0.001,
        input_dropout_ratio=0.1,
        hidden_dropout_ratios=[0.2, 0.2, 0.1],
        stopping_rounds=10,
        stopping_tolerance=0.001,
        stopping_metric="RMSE",
        nfolds=5,
        fold_assignment="Modulo",
        keep_cross_validation_predictions=True,
        seed=42
    )
    
    h2o_dl.train(
        x=h2o_predictors,
        y=TARGET,
        training_frame=h2o_train,
        validation_frame=h2o_test
    )
    
    dl_time = (datetime.now() - start_time).total_seconds()
    h2o_models['H2O_DL'] = h2o_dl
    
    # Métricas H2O Deep Learning
    train_rmse_dl = h2o_dl.rmse(train=True)
    valid_rmse_dl = h2o_dl.rmse(valid=True)
    train_r2_dl = h2o_dl.r2(train=True)
    valid_r2_dl = h2o_dl.r2(valid=True)
    
    h2o_results['H2O_DL'] = {
        'train_rmse': train_rmse_dl,
        'test_rmse': valid_rmse_dl,
        'train_r2': train_r2_dl,
        'test_r2': valid_r2_dl,
        'training_time': dl_time,
        'cv_rmse': h2o_dl.rmse(xval=True),
        'cv_r2': h2o_dl.r2(xval=True)
    }
    
    print(f"   Tiempo entrenamiento: {dl_time:.2f}s")
    print(f"   Train RMSE: {train_rmse_dl:,.0f}")
    print(f"   Test RMSE:  {valid_rmse_dl:,.0f}")
    print(f"   Test R²:    {valid_r2_dl:.4f}")
    print(f"   CV RMSE:    {h2o_dl.rmse(xval=True):,.0f}")
    
    # Resumen de modelos H2O
    print(f"\n📊 RESUMEN MODELOS H2O:")
    print("=" * 50)
    print(f"{'Modelo':<12} {'Test RMSE':<12} {'Test R²':<10} {'CV RMSE':<12} {'Tiempo':<10}")
    print("-" * 60)
    
    for model_name, results in h2o_results.items():
        print(f"{model_name:<12} {results['test_rmse']:<12,.0f} {results['test_r2']:<10.4f} "
              f"{results['cv_rmse']:<12,.0f} {results['training_time']:<10.1f}s")
    
    total_h2o_time = sum([r['training_time'] for r in h2o_results.values()])
    print(f"\n⏱️ Tiempo total H2O: {total_h2o_time:.1f}s")
    print(f"✅ Modelos H2O entrenados exitosamente")
    
else:
    print("⚠️ H2O no disponible, saltando modelos H2O")
    h2o_models = {}
    h2o_results = {}

In [ ]:
# ==================================================
# 2.2 H2O AUTOML - PIPELINE AUTOMATIZADO
# ==================================================

if H2O_AVAILABLE:
    print("🤖 Ejecutando H2O AutoML (Pipeline Automatizado)...")
    
    from h2o.automl import H2OAutoML
    
    start_time = datetime.now()
    
    # Configurar AutoML
    aml = H2OAutoML(
        max_models=20,  # Máximo 20 modelos
        max_runtime_secs=300,  # Máximo 5 minutos
        stopping_metric="RMSE",
        stopping_tolerance=0.001,
        stopping_rounds=3,
        seed=42,
        nfolds=5,
        keep_cross_validation_predictions=True,
        keep_cross_validation_models=True
    )
    
    # Entrenar AutoML
    aml.train(
        x=h2o_predictors,
        y=TARGET,
        training_frame=h2o_train,
        validation_frame=h2o_test
    )
    
    automl_time = (datetime.now() - start_time).total_seconds()
    
    print(f"   Tiempo AutoML: {automl_time:.1f}s")
    print(f"   Modelos entrenados: {len(aml.leaderboard)}")
    
    # Obtener el mejor modelo
    best_model = aml.leader
    print(f"   Mejor modelo: {best_model.model_id}")
    
    # Métricas del mejor modelo
    best_train_rmse = best_model.rmse(train=True)
    best_valid_rmse = best_model.rmse(valid=True)
    best_train_r2 = best_model.r2(train=True)
    best_valid_r2 = best_model.r2(valid=True)
    
    h2o_models['H2O_AutoML_Best'] = best_model
    h2o_results['H2O_AutoML_Best'] = {
        'train_rmse': best_train_rmse,
        'test_rmse': best_valid_rmse,
        'train_r2': best_train_r2,
        'test_r2': best_valid_r2,
        'training_time': automl_time,
        'cv_rmse': best_model.rmse(xval=True),
        'cv_r2': best_model.r2(xval=True),
        'model_type': type(best_model).__name__
    }
    
    print(f"   Best Train RMSE: {best_train_rmse:,.0f}")
    print(f"   Best Test RMSE:  {best_valid_rmse:,.0f}")
    print(f"   Best Test R²:    {best_valid_r2:.4f}")
    print(f"   Best CV RMSE:    {best_model.rmse(xval=True):,.0f}")
    
    # Mostrar leaderboard
    print(f"\n📋 LEADERBOARD H2O AutoML (Top 10):")
    print("-" * 80)
    leaderboard = aml.leaderboard.as_data_frame()
    print(leaderboard.head(10).to_string(index=False))
    
    # Guardar AutoML para análisis posterior
    h2o_models['H2O_AutoML'] = aml
    
    print(f"\n✅ H2O AutoML completado exitosamente")
    
else:
    print("⚠️ H2O no disponible, saltando AutoML")

In [ ]:
# ==================================================
# 2.1 MODELOS ESTADÍSTICOS OPTIMIZADOS (PARALELO)
# ==================================================

print("📊 Implementando modelos estadísticos con paralelización...")

# Preparar datos - escalado para modelos regularizados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Diccionario para almacenar modelos y resultados
models = {}
model_results = {}
statistical_models = {}

# 1. Regresión Lineal (OLS) - Baseline
print("\n🔸 Entrenando Regresión Lineal (OLS)...")
start_time = datetime.now()

lr = LinearRegression(n_jobs=-1)  # Usar todos los cores
lr.fit(X_train, y_train)
models['Linear_Regression'] = lr

# Predicciones
y_pred_train_lr = lr.predict(X_train)
y_pred_test_lr = lr.predict(X_test)

# Métricas
lr_metrics = {
    'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train_lr)),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test_lr)),
    'train_mae': mean_absolute_error(y_train, y_pred_train_lr),
    'test_mae': mean_absolute_error(y_test, y_pred_test_lr),
    'train_r2': r2_score(y_train, y_pred_train_lr),
    'test_r2': r2_score(y_test, y_pred_test_lr),
    'training_time': (datetime.now() - start_time).total_seconds()
}
model_results['Linear_Regression'] = lr_metrics
statistical_models['Linear_Regression'] = {'model': lr, **lr_metrics}

print(f"   Tiempo entrenamiento: {lr_metrics['training_time']:.2f}s")
print(f"   Train RMSE: {lr_metrics['train_rmse']:,.0f}")
print(f"   Test RMSE:  {lr_metrics['test_rmse']:,.0f}")
print(f"   Test R²:    {lr_metrics['test_r2']:.4f}")

# 2. Ridge Regression (Paralelizado)
print("\n🔸 Entrenando Ridge Regression (Paralelo)...")
start_time = datetime.now()

# Búsqueda paralela de hiperparámetros
from sklearn.model_selection import GridSearchCV

ridge_params = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}
ridge_grid = GridSearchCV(
    Ridge(random_state=42), 
    ridge_params, 
    cv=5, 
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,  # Usar todos los cores
    verbose=0
)
ridge_grid.fit(X_train_scaled, y_train)

ridge = ridge_grid.best_estimator_
models['Ridge'] = ridge

y_pred_train_ridge = ridge.predict(X_train_scaled)
y_pred_test_ridge = ridge.predict(X_test_scaled)

ridge_metrics = {
    'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train_ridge)),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test_ridge)),
    'train_mae': mean_absolute_error(y_train, y_pred_train_ridge),
    'test_mae': mean_absolute_error(y_test, y_pred_test_ridge),
    'train_r2': r2_score(y_train, y_pred_train_ridge),
    'test_r2': r2_score(y_test, y_pred_test_ridge),
    'best_alpha': ridge_grid.best_params_['alpha'],
    'training_time': (datetime.now() - start_time).total_seconds()
}
model_results['Ridge'] = ridge_metrics
statistical_models['Ridge'] = {'model': ridge, **ridge_metrics}

print(f"   Tiempo entrenamiento: {ridge_metrics['training_time']:.2f}s")
print(f"   Alpha óptimo: {ridge_metrics['best_alpha']}")
print(f"   Train RMSE: {ridge_metrics['train_rmse']:,.0f}")
print(f"   Test RMSE:  {ridge_metrics['test_rmse']:,.0f}")
print(f"   Test R²:    {ridge_metrics['test_r2']:.4f}")

# 3. Lasso Regression (Paralelizado)
print("\n🔸 Entrenando Lasso Regression (Paralelo)...")
start_time = datetime.now()

lasso_params = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}
lasso_grid = GridSearchCV(
    Lasso(random_state=42, max_iter=3000), 
    lasso_params, 
    cv=5, 
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,  # Usar todos los cores
    verbose=0
)
lasso_grid.fit(X_train_scaled, y_train)

lasso = lasso_grid.best_estimator_
models['Lasso'] = lasso

y_pred_train_lasso = lasso.predict(X_train_scaled)
y_pred_test_lasso = lasso.predict(X_test_scaled)

# Contar features seleccionadas por Lasso
n_selected_features = np.sum(lasso.coef_ != 0)

lasso_metrics = {
    'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train_lasso)),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test_lasso)),
    'train_mae': mean_absolute_error(y_train, y_pred_train_lasso),
    'test_mae': mean_absolute_error(y_test, y_pred_test_lasso),
    'train_r2': r2_score(y_train, y_pred_train_lasso),
    'test_r2': r2_score(y_test, y_pred_test_lasso),
    'best_alpha': lasso_grid.best_params_['alpha'],
    'n_features': n_selected_features,
    'training_time': (datetime.now() - start_time).total_seconds()
}
model_results['Lasso'] = lasso_metrics
statistical_models['Lasso'] = {'model': lasso, **lasso_metrics}

print(f"   Tiempo entrenamiento: {lasso_metrics['training_time']:.2f}s")
print(f"   Alpha óptimo: {lasso_metrics['best_alpha']}")
print(f"   Features seleccionadas: {n_selected_features}/{len(available_features)}")
print(f"   Train RMSE: {lasso_metrics['train_rmse']:,.0f}")
print(f"   Test RMSE:  {lasso_metrics['test_rmse']:,.0f}")
print(f"   Test R²:    {lasso_metrics['test_r2']:.4f}")

# 4. ElasticNet (Paralelizado)
print("\n🔸 Entrenando ElasticNet (Paralelo)...")
start_time = datetime.now()

elastic_params = {
    'alpha': [0.01, 0.1, 1.0, 10.0],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
}
elastic_grid = GridSearchCV(
    ElasticNet(random_state=42, max_iter=3000), 
    elastic_params, 
    cv=5, 
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,  # Usar todos los cores
    verbose=0
)
elastic_grid.fit(X_train_scaled, y_train)

elastic = elastic_grid.best_estimator_
models['ElasticNet'] = elastic

y_pred_train_elastic = elastic.predict(X_train_scaled)
y_pred_test_elastic = elastic.predict(X_test_scaled)

n_selected_features_elastic = np.sum(elastic.coef_ != 0)

elastic_metrics = {
    'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train_elastic)),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test_elastic)),
    'train_mae': mean_absolute_error(y_train, y_pred_train_elastic),
    'test_mae': mean_absolute_error(y_test, y_pred_test_elastic),
    'train_r2': r2_score(y_train, y_pred_train_elastic),
    'test_r2': r2_score(y_test, y_pred_test_elastic),
    'best_alpha': elastic_grid.best_params_['alpha'],
    'best_l1_ratio': elastic_grid.best_params_['l1_ratio'],
    'n_features': n_selected_features_elastic,
    'training_time': (datetime.now() - start_time).total_seconds()
}
model_results['ElasticNet'] = elastic_metrics
statistical_models['ElasticNet'] = {'model': elastic, **elastic_metrics}

print(f"   Tiempo entrenamiento: {elastic_metrics['training_time']:.2f}s")
print(f"   Alpha óptimo: {elastic_metrics['best_alpha']}")
print(f"   L1 ratio óptimo: {elastic_metrics['best_l1_ratio']}")
print(f"   Features seleccionadas: {n_selected_features_elastic}/{len(available_features)}")
print(f"   Train RMSE: {elastic_metrics['train_rmse']:,.0f}")
print(f"   Test RMSE:  {elastic_metrics['test_rmse']:,.0f}")
print(f"   Test R²:    {elastic_metrics['test_r2']:.4f}")

# Guardar modelos estadísticos
model_results['statistical_models'] = statistical_models

print(f"\n✅ Modelos estadísticos entrenados correctamente con paralelización")
print(f"⏱️ Tiempo total modelos estadísticos: {sum([m['training_time'] for m in model_results.values() if 'training_time' in m]):.2f}s")

In [ ]:
# ==================================================
# 2.2 COMPARACIÓN CON AIC/BIC (SIN CV)
# ==================================================

print("🔍 Calculando AIC/BIC para comparación de modelos...")

def calculate_aic_bic(y_true, y_pred, n_params, n_obs):
    """Calcula AIC y BIC para un modelo"""
    # Suma de cuadrados de residuos
    ssr = np.sum((y_true - y_pred) ** 2)
    
    # Log-likelihood (asumiendo errores normales)
    sigma2 = ssr / n_obs
    log_likelihood = -0.5 * n_obs * (np.log(2 * np.pi) + np.log(sigma2) + 1)
    
    # AIC y BIC
    aic = 2 * n_params - 2 * log_likelihood
    bic = np.log(n_obs) * n_params - 2 * log_likelihood
    
    return aic, bic

# Calcular AIC/BIC para cada modelo estadístico
n_obs = len(y_train)
model_comparison = []

# Linear Regression
n_params_lr = len(available_features) + 1  # features + intercept
aic_lr, bic_lr = calculate_aic_bic(y_train, y_pred_train_lr, n_params_lr, n_obs)

model_comparison.append({
    'Model': 'Linear Regression',
    'AIC': aic_lr,
    'BIC': bic_lr,
    'N_Params': n_params_lr,
    'Test_R2': lr_metrics['test_r2'],
    'Test_RMSE': lr_metrics['test_rmse']
})

# Ridge (todos los parámetros excepto regularización)
n_params_ridge = len(available_features) + 1
aic_ridge, bic_ridge = calculate_aic_bic(y_train, y_pred_train_ridge, n_params_ridge, n_obs)

model_comparison.append({
    'Model': 'Ridge',
    'AIC': aic_ridge,
    'BIC': bic_ridge,
    'N_Params': n_params_ridge,
    'Test_R2': ridge_metrics['test_r2'],
    'Test_RMSE': ridge_metrics['test_rmse']
})

# Lasso (solo parámetros no-cero)
n_params_lasso = n_selected_features + 1
aic_lasso, bic_lasso = calculate_aic_bic(y_train, y_pred_train_lasso, n_params_lasso, n_obs)

model_comparison.append({
    'Model': 'Lasso',
    'AIC': aic_lasso,
    'BIC': bic_lasso,
    'N_Params': n_params_lasso,
    'Test_R2': lasso_metrics['test_r2'],
    'Test_RMSE': lasso_metrics['test_rmse']
})

# ElasticNet (solo parámetros no-cero)
n_params_elastic = n_selected_features_elastic + 1
aic_elastic, bic_elastic = calculate_aic_bic(y_train, y_pred_train_elastic, n_params_elastic, n_obs)

model_comparison.append({
    'Model': 'ElasticNet',
    'AIC': aic_elastic,
    'BIC': bic_elastic,
    'N_Params': n_params_elastic,
    'Test_R2': elastic_metrics['test_r2'],
    'Test_RMSE': elastic_metrics['test_rmse']
})

# Crear DataFrame para comparación
comparison_df = pd.DataFrame(model_comparison)
comparison_df = comparison_df.sort_values('AIC').reset_index(drop=True)

print("📊 Comparación de modelos estadísticos (ordenado por AIC):")
print("=" * 80)
print(f"{'Modelo':<15} {'AIC':<12} {'BIC':<12} {'N_Params':<10} {'Test_R²':<10} {'Test_RMSE':<12}")
print("-" * 80)

for _, row in comparison_df.iterrows():
    print(f"{row['Model']:<15} {row['AIC']:<12,.0f} {row['BIC']:<12,.0f} {row['N_Params']:<10} {row['Test_R2']:<10.4f} {row['Test_RMSE']:<12,.0f}")

# Identificar mejor modelo por cada criterio
best_aic = comparison_df.loc[comparison_df['AIC'].idxmin(), 'Model']
best_bic = comparison_df.loc[comparison_df['BIC'].idxmin(), 'Model']
best_r2 = comparison_df.loc[comparison_df['Test_R2'].idxmax(), 'Model']
best_rmse = comparison_df.loc[comparison_df['Test_RMSE'].idxmin(), 'Model']

print(f"\n🏆 Mejores modelos por criterio:")
print(f"   Mejor AIC:    {best_aic}")
print(f"   Mejor BIC:    {best_bic}")
print(f"   Mejor R²:     {best_r2}")
print(f"   Mejor RMSE:   {best_rmse}")

# Visualización de comparación
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# AIC vs BIC
axes[0].scatter(comparison_df['AIC'], comparison_df['BIC'], s=100, alpha=0.7)
for i, row in comparison_df.iterrows():
    axes[0].annotate(row['Model'], (row['AIC'], row['BIC']), 
                    xytext=(5, 5), textcoords='offset points', fontsize=10)
axes[0].set_xlabel('AIC')
axes[0].set_ylabel('BIC')
axes[0].set_title('AIC vs BIC')
axes[0].grid(True, alpha=0.3)

# Número de parámetros vs Performance
axes[1].scatter(comparison_df['N_Params'], comparison_df['Test_R2'], s=100, alpha=0.7, color='green')
for i, row in comparison_df.iterrows():
    axes[1].annotate(row['Model'], (row['N_Params'], row['Test_R2']), 
                    xytext=(5, 5), textcoords='offset points', fontsize=10)
axes[1].set_xlabel('Número de Parámetros')
axes[1].set_ylabel('Test R²')
axes[1].set_title('Complejidad vs Performance')
axes[1].grid(True, alpha=0.3)

# Ranking de modelos
ranking_criteria = ['AIC', 'BIC', 'Test_R2', 'Test_RMSE']
model_names = comparison_df['Model'].tolist()
rankings = []

for criterion in ranking_criteria:
    if criterion in ['AIC', 'BIC', 'Test_RMSE']:
        # Menor es mejor
        sorted_df = comparison_df.sort_values(criterion)
    else:
        # Mayor es mejor
        sorted_df = comparison_df.sort_values(criterion, ascending=False)
    
    ranking = {model: idx+1 for idx, model in enumerate(sorted_df['Model'])}
    rankings.append([ranking[model] for model in model_names])

# Heatmap de rankings
ranking_matrix = np.array(rankings).T
im = axes[2].imshow(ranking_matrix, cmap='RdYlGn_r', aspect='auto')
axes[2].set_xticks(range(len(ranking_criteria)))
axes[2].set_xticklabels(ranking_criteria)
axes[2].set_yticks(range(len(model_names)))
axes[2].set_yticklabels(model_names)
axes[2].set_title('Ranking de Modelos (1=mejor)')

# Añadir valores al heatmap
for i in range(len(model_names)):
    for j in range(len(ranking_criteria)):
        axes[2].text(j, i, int(ranking_matrix[i, j]), ha='center', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Guardar resultados
model_results['comparison_df'] = comparison_df

In [ ]:
# ==================================================
# 2.3 DIAGNÓSTICO DE RESIDUOS Y DETECCIÓN DE OUTLIERS
# ==================================================

print("🔍 Realizando diagnóstico de residuos para el mejor modelo...")

# Usar el mejor modelo según AIC (Linear Regression en este caso)
best_model_name = best_aic
print(f"Analizando residuos para: {best_model_name}")

# Obtener predicciones del mejor modelo
if best_model_name == 'Linear_Regression':
    y_pred_train_best = y_pred_train_lr
    y_pred_test_best = y_pred_test_lr
elif best_model_name == 'Ridge':
    y_pred_train_best = y_pred_train_ridge
    y_pred_test_best = y_pred_test_ridge
elif best_model_name == 'Lasso':
    y_pred_train_best = y_pred_train_lasso
    y_pred_test_best = y_pred_test_lasso
else:  # ElasticNet
    y_pred_train_best = y_pred_train_elastic
    y_pred_test_best = y_pred_test_elastic

# Calcular residuos
residuals_train = y_train - y_pred_train_best
residuals_test = y_test - y_pred_test_best

# Residuos estandarizados
residuals_train_std = residuals_train / np.std(residuals_train)
residuals_test_std = residuals_test / np.std(residuals_test)

def diagnostic_plots(y_true, y_pred, residuals, residuals_std, title_prefix):
    """Crea gráficos de diagnóstico de residuos"""
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Residuos vs Fitted Values
    axes[0,0].scatter(y_pred, residuals, alpha=0.5)
    axes[0,0].axhline(y=0, color='red', linestyle='--')
    axes[0,0].set_xlabel('Valores Predichos')
    axes[0,0].set_ylabel('Residuos')
    axes[0,0].set_title(f'{title_prefix}: Residuos vs Predicciones')
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Q-Q Plot de residuos
    stats.probplot(residuals, dist="norm", plot=axes[0,1])
    axes[0,1].set_title(f'{title_prefix}: Q-Q Plot Residuos')
    axes[0,1].grid(True, alpha=0.3)
    
    # 3. Histograma de residuos
    axes[0,2].hist(residuals, bins=50, density=True, alpha=0.7, color='skyblue')
    axes[0,2].set_xlabel('Residuos')
    axes[0,2].set_ylabel('Densidad')
    axes[0,2].set_title(f'{title_prefix}: Distribución de Residuos')
    axes[0,2].grid(True, alpha=0.3)
    
    # 4. Scale-Location Plot (Residuos estandarizados vs Fitted)
    axes[1,0].scatter(y_pred, np.abs(residuals_std), alpha=0.5)
    axes[1,0].set_xlabel('Valores Predichos')
    axes[1,0].set_ylabel('|Residuos Estandarizados|')
    axes[1,0].set_title(f'{title_prefix}: Scale-Location Plot')
    axes[1,0].grid(True, alpha=0.3)
    
    # 5. Residuos vs Orden (para detectar autocorrelación)
    axes[1,1].plot(range(len(residuals)), residuals, alpha=0.7)
    axes[1,1].axhline(y=0, color='red', linestyle='--')
    axes[1,1].set_xlabel('Orden de Observación')
    axes[1,1].set_ylabel('Residuos')
    axes[1,1].set_title(f'{title_prefix}: Residuos vs Orden')
    axes[1,1].grid(True, alpha=0.3)
    
    # 6. Predicted vs Actual
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    axes[1,2].scatter(y_true, y_pred, alpha=0.5)
    axes[1,2].plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    axes[1,2].set_xlabel('Valores Reales')
    axes[1,2].set_ylabel('Valores Predichos')
    axes[1,2].set_title(f'{title_prefix}: Predicho vs Real')
    axes[1,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Diagnósticos para conjunto de entrenamiento
print("\n📊 Diagnósticos para conjunto de ENTRENAMIENTO:")
diagnostic_plots(y_train, y_pred_train_best, residuals_train, residuals_train_std, "Train")

# Diagnósticos para conjunto de prueba
print("\n📊 Diagnósticos para conjunto de PRUEBA:")
diagnostic_plots(y_test, y_pred_test_best, residuals_test, residuals_test_std, "Test")

# Tests estadísticos de los residuos
print("\n🧪 Tests estadísticos de residuos:")

# Test de normalidad (Shapiro-Wilk en muestra pequeña)
from scipy.stats import shapiro, jarque_bera

# Tomar muestra para test de normalidad (Shapiro-Wilk max 5000 observaciones)
sample_size = min(5000, len(residuals_train))
residuals_sample = np.random.choice(residuals_train, sample_size, replace=False)

shapiro_stat, shapiro_p = shapiro(residuals_sample)
jb_stat, jb_p = jarque_bera(residuals_train)

print(f"Test de normalidad:")
print(f"  Shapiro-Wilk (muestra {sample_size}): stat={shapiro_stat:.4f}, p-value={shapiro_p:.4f}")
print(f"  Jarque-Bera: stat={jb_stat:.4f}, p-value={jb_p:.4f}")

# Test de heterocedasticidad (Breusch-Pagan)
if best_model_name == 'Linear_Regression':
    # Usar statsmodels para test más riguroso
    X_train_sm = sm.add_constant(X_train)
    model_sm = sm.OLS(y_train, X_train_sm).fit()
    
    # Test de Breusch-Pagan
    from statsmodels.stats.diagnostic import het_breuschpagan
    bp_stat, bp_p, _, _ = het_breuschpagan(model_sm.resid, X_train_sm)
    
    print(f"\nTest de heterocedasticidad:")
    print(f"  Breusch-Pagan: stat={bp_stat:.4f}, p-value={bp_p:.4f}")
    
    if bp_p < 0.05:
        print("  ⚠️ Evidencia de heterocedasticidad")
    else:
        print("  ✅ No hay evidencia de heterocedasticidad")

# Detección de outliers
print(f"\n🎯 Detección de outliers:")

# Outliers basados en residuos estandarizados (|z| > 3)
outliers_train = np.where(np.abs(residuals_train_std) > 3)[0]
outliers_test = np.where(np.abs(residuals_test_std) > 3)[0]

print(f"Outliers por residuos estandarizados (|z| > 3):")
print(f"  Train: {len(outliers_train)} outliers ({len(outliers_train)/len(y_train)*100:.2f}%)")
print(f"  Test:  {len(outliers_test)} outliers ({len(outliers_test)/len(y_test)*100:.2f}%)")

# Estadísticas de los outliers
if len(outliers_train) > 0:
    outlier_residuals = residuals_train_std[outliers_train]
    print(f"\nEstadísticas de outliers (train):")
    print(f"  Residuo estandarizado máximo: {np.max(np.abs(outlier_residuals)):.2f}")
    print(f"  Residuo estandarizado promedio: {np.mean(np.abs(outlier_residuals)):.2f}")

# Análisis de leverage y influencia (solo para Linear Regression)
if best_model_name == 'Linear_Regression':
    print(f"\n📈 Análisis de leverage e influencia:")
    
    # Calcular leverage, residuos estudentizados y distancia de Cook
    influence = OLSInfluence(model_sm)
    leverage = influence.hat_matrix_diag
    studentized_residuals = influence.resid_studentized_external
    cooks_distance = influence.cooks_distance[0]
    
    # Umbrales
    leverage_threshold = 2 * (len(available_features) + 1) / len(y_train)
    cooks_threshold = 4 / len(y_train)
    
    # Identificar observaciones influyentes
    high_leverage = np.where(leverage > leverage_threshold)[0]
    high_cooks = np.where(cooks_distance > cooks_threshold)[0]
    high_studentized = np.where(np.abs(studentized_residuals) > 3)[0]
    
    print(f"  High leverage (>{leverage_threshold:.4f}): {len(high_leverage)} obs ({len(high_leverage)/len(y_train)*100:.2f}%)")
    print(f"  High Cook's distance (>{cooks_threshold:.4f}): {len(high_cooks)} obs ({len(high_cooks)/len(y_train)*100:.2f}%)")
    print(f"  High studentized residuals (>3): {len(high_studentized)} obs ({len(high_studentized)/len(y_train)*100:.2f}%)")
    
    # Observaciones problemáticas (combinan múltiples criterios)
    problematic = np.intersect1d(high_leverage, high_cooks)
    if len(problematic) > 0:
        print(f"  ⚠️ Observaciones problemáticas (alto leverage Y Cook's distance): {len(problematic)}")
    else:
        print(f"  ✅ No hay observaciones extremadamente problemáticas")

print(f"\n✅ Diagnóstico de residuos completado")

## 3. Modelos de Árboles

### Metodología de Modelos Basados en Árboles
- **LightGBM**: Gradient boosting eficiente con parámetros por defecto
- **Random Forest**: Ensemble de árboles con configuración básica
- **Comparación**: Performance vs modelos estadísticos
- **Interpretabilidad**: Feature importance y explicabilidad inicial

In [ ]:
# ==================================================
# 3.1 MODELOS DE ÁRBOLES OPTIMIZADOS (GPU + PARALELO)
# ==================================================

print("🌳 Entrenando modelos basados en árboles con GPU y paralelización...")

# 1. LightGBM con GPU (si disponible)
print("\n🔸 Entrenando LightGBM (GPU Optimizado)...")
start_time = datetime.now()

# Parámetros optimizados para GPU y paralelización
if GPU_AVAILABLE:
    lgb_params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'num_leaves': 63,  # Más hojas para GPU
        'learning_rate': 0.1,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'max_depth': 7,
        'min_data_in_leaf': 20,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'verbose': -1,
        'random_state': 42,
        'force_col_wise': True,  # Optimización para GPU
        'gpu_use_dp': False  # Usar single precision para velocidad
    }
    print("   🚀 Usando GPU para entrenamiento")
else:
    lgb_params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'device': 'cpu',
        'num_threads': mp.cpu_count(),  # Usar todos los cores
        'num_leaves': 31,
        'learning_rate': 0.1,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'random_state': 42,
        'force_col_wise': True
    }
    print(f"   ⚡ Usando CPU con {mp.cpu_count()} threads")

# Crear datasets de LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Entrenar modelo con early stopping optimizado
lgb_model = lgb.train(
    lgb_params,
    train_data,
    num_boost_round=500,  # Más iteraciones para mejor rendimiento
    valid_sets=[train_data, valid_data],
    valid_names=['train', 'valid'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=20),
        lgb.log_evaluation(50)  # Log cada 50 iteraciones
    ]
)

training_time_lgb = (datetime.now() - start_time).total_seconds()

# Predicciones
y_pred_train_lgb = lgb_model.predict(X_train, num_iteration=lgb_model.best_iteration)
y_pred_test_lgb = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)

# Métricas LightGBM
lgb_metrics = {
    'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train_lgb)),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test_lgb)),
    'train_mae': mean_absolute_error(y_train, y_pred_train_lgb),
    'test_mae': mean_absolute_error(y_test, y_pred_test_lgb),
    'train_r2': r2_score(y_train, y_pred_train_lgb),
    'test_r2': r2_score(y_test, y_pred_test_lgb),
    'best_iteration': lgb_model.best_iteration,
    'num_features': lgb_model.num_feature(),
    'training_time': training_time_lgb,
    'device': 'GPU' if GPU_AVAILABLE else 'CPU'
}

models['LightGBM'] = lgb_model
model_results['LightGBM'] = lgb_metrics

print(f"   Tiempo entrenamiento: {training_time_lgb:.2f}s")
print(f"   Mejores iteraciones: {lgb_model.best_iteration}")
print(f"   Device usado: {lgb_metrics['device']}")
print(f"   Train RMSE: {lgb_metrics['train_rmse']:,.0f}")
print(f"   Test RMSE:  {lgb_metrics['test_rmse']:,.0f}")
print(f"   Test R²:    {lgb_metrics['test_r2']:.4f}")

# 2. Random Forest con paralelización optimizada
print("\n🔸 Entrenando Random Forest (Paralelo Optimizado)...")
start_time = datetime.now()

# Parámetros optimizados para paralelización
rf_params = {
    'n_estimators': 200,  # Más árboles para mejor rendimiento
    'max_depth': 15,  # Más profundidad
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'bootstrap': True,
    'oob_score': True,  # Out-of-bag score para validación
    'random_state': 42,
    'n_jobs': -1,  # Usar todos los cores
    'verbose': 1  # Mostrar progreso
}

rf_model = RandomForestRegressor(**rf_params)
rf_model.fit(X_train, y_train)

training_time_rf = (datetime.now() - start_time).total_seconds()

# Predicciones
y_pred_train_rf = rf_model.predict(X_train)
y_pred_test_rf = rf_model.predict(X_test)

# Métricas Random Forest
rf_metrics = {
    'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train_rf)),
    'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test_rf)),
    'train_mae': mean_absolute_error(y_train, y_pred_train_rf),
    'test_mae': mean_absolute_error(y_test, y_pred_test_rf),
    'train_r2': r2_score(y_train, y_pred_train_rf),
    'test_r2': r2_score(y_test, y_pred_test_rf),
    'oob_score': rf_model.oob_score_,
    'n_estimators': rf_params['n_estimators'],
    'max_depth': rf_params['max_depth'],
    'training_time': training_time_rf,
    'n_cores': mp.cpu_count()
}

models['Random_Forest'] = rf_model
model_results['Random_Forest'] = rf_metrics

print(f"   Tiempo entrenamiento: {training_time_rf:.2f}s")
print(f"   Número de árboles: {rf_params['n_estimators']}")
print(f"   Cores utilizados: {mp.cpu_count()}")
print(f"   OOB Score: {rf_metrics['oob_score']:.4f}")
print(f"   Train RMSE: {rf_metrics['train_rmse']:,.0f}")
print(f"   Test RMSE:  {rf_metrics['test_rmse']:,.0f}")
print(f"   Test R²:    {rf_metrics['test_r2']:.4f}")

# Comparación de velocidad de entrenamiento
print(f"\n⚡ COMPARACIÓN DE VELOCIDAD:")
print(f"   LightGBM ({lgb_metrics['device']}): {training_time_lgb:.2f}s")
print(f"   Random Forest ({mp.cpu_count()} cores): {training_time_rf:.2f}s")
print(f"   Speedup LGB vs RF: {training_time_rf/training_time_lgb:.1f}x")

print(f"\n✅ Modelos de árboles entrenados correctamente con optimizaciones")

In [ ]:
# ==================================================
# 3.2 FEATURE IMPORTANCE Y EXPLICABILIDAD INICIAL
# ==================================================

print("🔍 Analizando feature importance de modelos de árboles...")

def plot_feature_importance(model, model_name, feature_names, top_n=15):
    """Grafica feature importance para modelos de árboles"""
    
    if model_name == 'LightGBM':
        # Feature importance de LightGBM
        importance = model.feature_importance(importance_type='gain')
        feature_imp = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)
        
    elif model_name == 'Random_Forest':
        # Feature importance de Random Forest
        importance = model.feature_importances_
        feature_imp = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)
    
    # Plot top features
    top_features = feature_imp.head(top_n)
    
    plt.figure(figsize=(10, 8))
    plt.barh(range(len(top_features)), top_features['importance'], color='lightcoral', alpha=0.8)
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'Top {top_n} Features - {model_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    return feature_imp

# Analizar feature importance para LightGBM
print(f"\n📊 Feature Importance - LightGBM:")
lgb_feature_imp = plot_feature_importance(lgb_model, 'LightGBM', available_features)

# Mostrar top 10 features de LightGBM
print(f"Top 10 Features más importantes (LightGBM):")
for i, (_, row) in enumerate(lgb_feature_imp.head(10).iterrows(), 1):
    print(f"  {i:2d}. {row['feature']:<25}: {row['importance']:.2f}")

# Analizar feature importance para Random Forest
print(f"\n📊 Feature Importance - Random Forest:")
rf_feature_imp = plot_feature_importance(rf_model, 'Random_Forest', available_features)

# Mostrar top 10 features de Random Forest
print(f"Top 10 Features más importantes (Random Forest):")
for i, (_, row) in enumerate(rf_feature_imp.head(10).iterrows(), 1):
    print(f"  {i:2d}. {row['feature']:<25}: {row['importance']:.4f}")

# Comparación de feature importance entre modelos de árboles
print(f"\n🔄 Comparación de Feature Importance entre modelos:")

# Merge de importancias
comparison_imp = lgb_feature_imp[['feature', 'importance']].rename(columns={'importance': 'lgb_importance'})
comparison_imp = comparison_imp.merge(
    rf_feature_imp[['feature', 'importance']].rename(columns={'importance': 'rf_importance'}),
    on='feature'
)

# Normalizar para comparación
comparison_imp['lgb_importance_norm'] = comparison_imp['lgb_importance'] / comparison_imp['lgb_importance'].max()
comparison_imp['rf_importance_norm'] = comparison_imp['rf_importance'] / comparison_imp['rf_importance'].max()

# Calcular correlación entre importancias
correlation = comparison_imp['lgb_importance_norm'].corr(comparison_imp['rf_importance_norm'])
print(f"Correlación entre importancias LightGBM-RF: {correlation:.4f}")

# Plot comparativo de importancias
top_features_comparison = comparison_imp.head(15)

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Scatter plot de importancias
axes[0].scatter(comparison_imp['lgb_importance_norm'], comparison_imp['rf_importance_norm'], alpha=0.6)
axes[0].plot([0, 1], [0, 1], 'r--', alpha=0.8)
axes[0].set_xlabel('LightGBM Importance (normalized)')
axes[0].set_ylabel('Random Forest Importance (normalized)')
axes[0].set_title(f'Comparación Feature Importance\n(Correlación: {correlation:.3f})')
axes[0].grid(True, alpha=0.3)

# Top features side by side
x_pos = np.arange(len(top_features_comparison))
width = 0.35

axes[1].barh(x_pos - width/2, top_features_comparison['lgb_importance_norm'], width, 
            label='LightGBM', alpha=0.8, color='lightblue')
axes[1].barh(x_pos + width/2, top_features_comparison['rf_importance_norm'], width,
            label='Random Forest', alpha=0.8, color='lightcoral')

axes[1].set_yticks(x_pos)
axes[1].set_yticklabels(top_features_comparison['feature'])
axes[1].set_xlabel('Normalized Importance')
axes[1].set_title('Top 15 Features: LightGBM vs Random Forest')
axes[1].legend()
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

# Features que aparecen en top 10 de ambos modelos
lgb_top10 = set(lgb_feature_imp.head(10)['feature'])
rf_top10 = set(rf_feature_imp.head(10)['feature'])
common_top10 = lgb_top10.intersection(rf_top10)

print(f"\n🎯 Features en Top 10 de ambos modelos ({len(common_top10)}):")
for feature in sorted(common_top10):
    lgb_rank = lgb_feature_imp[lgb_feature_imp['feature'] == feature].index[0] + 1
    rf_rank = rf_feature_imp[rf_feature_imp['feature'] == feature].index[0] + 1
    print(f"  • {feature} (LGB: #{lgb_rank}, RF: #{rf_rank})")

# Guardar feature importance
model_results['lgb_feature_importance'] = lgb_feature_imp
model_results['rf_feature_importance'] = rf_feature_imp
model_results['feature_importance_correlation'] = correlation

print(f"\n✅ Análisis de feature importance completado")

## 4. Evaluación Comparativa y Selección de Modelos

### 4.1 Comparación Global de Modelos

En esta sección realizaremos una evaluación comparativa sistemática de todos los modelos desarrollados, considerando múltiples criterios de evaluación y el contexto específico del problema de predicción de precios inmobiliarios.

**Metodología de Evaluación:**
- **Métricas de Rendimiento**: RMSE, MAE, R² para evaluación cuantitativa
- **Criterios Estadísticos**: AIC/BIC para modelos lineales (penalización por complejidad)
- **Análisis de Residuos**: Distribución, heterocedasticidad, outliers
- **Estabilidad Temporal**: Robustez en datos de test (simulando nuevas observaciones)
- **Interpretabilidad**: Facilidad de explicación para stakeholders inmobiliarios

In [ ]:
# ==================================================
# 5.1 EVALUACIÓN COMPARATIVA H2O + SKLEARN + GPU
# ==================================================

print("📊 Evaluación comparativa completa: H2O + Sklearn + GPU...")

# Función para calcular métricas completas en paralelo
def evaluate_model_comprehensive(y_true, y_pred, model_name, n_params=None, training_time=None, cv_metrics=None):
    """Calcula métricas completas para evaluación de modelos"""
    
    # Métricas básicas calculadas en paralelo donde sea posible
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    # Métricas adicionales
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    residuals = y_true - y_pred
    std_residuals = np.std(residuals)
    
    # Error relativo al rango
    price_range = np.max(y_true) - np.min(y_true)
    rmse_relative = (rmse / price_range) * 100
    
    # Crear diccionario de métricas
    metrics = {
        'model': model_name,
        'rmse': rmse,
        'mae': mae,
        'r2': r2,
        'mape': mape,
        'std_residuals': std_residuals,
        'rmse_relative': rmse_relative,
        'n_params': n_params,
        'training_time': training_time,
        'framework': 'H2O' if model_name.startswith('H2O') else 'Sklearn'
    }
    
    # Añadir métricas de validación cruzada si están disponibles
    if cv_metrics:
        metrics.update(cv_metrics)
    
    return metrics

# Evaluar todos los modelos usando paralelización donde sea posible
print("🎯 Evaluando todos los modelos (H2O + Sklearn + GPU)...")

evaluation_results = []

# === MODELOS H2O ===
if H2O_AVAILABLE and h2o_results:
    print("🚀 Evaluando modelos H2O...")
    
    for model_name, model_info in h2o_results.items():
        # Para H2O usamos métricas ya calculadas
        metrics = {
            'model': model_name,
            'rmse': model_info['test_rmse'],
            'mae': model_info['test_rmse'] * 0.7,  # Aproximación MAE ≈ 0.7 * RMSE
            'r2': model_info['test_r2'],
            'mape': (model_info['test_rmse'] / y_test.mean()) * 100,  # Aproximación
            'training_time': model_info['training_time'],
            'cv_rmse': model_info.get('cv_rmse', None),
            'cv_r2': model_info.get('cv_r2', None),
            'framework': 'H2O',
            'distributed': True
        }
        
        evaluation_results.append(metrics)

# === MODELOS SKLEARN ESTADÍSTICOS ===
if 'statistical_models' in model_results:
    print("📊 Evaluando modelos Sklearn...")
    
    for model_name, model_info in model_results['statistical_models'].items():
        model = model_info['model']
        
        # Predicciones optimizadas
        if hasattr(model, 'n_jobs'):
            model.n_jobs = -1  # Asegurar paralelización
        
        if model_name in ['Ridge', 'Lasso', 'ElasticNet']:
            y_pred = model.predict(X_test_scaled)
        else:
            y_pred = model.predict(X_test)
        
        # Número de parámetros para AIC/BIC
        if hasattr(model, 'coef_'):
            if model_name in ['Lasso', 'ElasticNet']:
                n_params = np.sum(model.coef_ != 0) + 1  # Solo parámetros no-cero + intercept
            else:
                n_params = len(model.coef_) + 1  # Todos los parámetros + intercept
        else:
            n_params = X_test.shape[1] + 1  # Features + intercept
        
        metrics = evaluate_model_comprehensive(
            y_test, y_pred, model_name, n_params, 
            model_info.get('training_time', None)
        )
        
        evaluation_results.append(metrics)

# === MODELOS DE ÁRBOLES ===
tree_models_info = {
    'LightGBM': (lgb_model, lambda m: m.predict(X_test, num_iteration=m.best_iteration)),
    'Random_Forest': (rf_model, lambda m: m.predict(X_test))
}

for model_name, (model, predict_fn) in tree_models_info.items():
    if model_name in model_results:
        model_info = model_results[model_name]
        y_pred = predict_fn(model)
        
        if model_name == 'LightGBM':
            n_params = model.best_iteration * len(available_features_sklearn)
        else:
            n_params = model.n_estimators * len(available_features_sklearn)
        
        metrics = evaluate_model_comprehensive(
            y_test, y_pred, model_name, n_params, 
            model_info.get('training_time', None)
        )
        
        # Añadir métricas específicas
        if model_name == 'LightGBM':
            metrics['device'] = model_info.get('device', 'CPU')
            metrics['best_iteration'] = model_info.get('best_iteration', 0)
        else:
            metrics['oob_score'] = model_info.get('oob_score', None)
            metrics['n_cores'] = model_info.get('n_cores', 1)
        
        evaluation_results.append(metrics)

# Crear DataFrame de resultados optimizado
results_df = pd.DataFrame(evaluation_results)

# Mostrar tabla comparativa completa
print("\n📋 RESUMEN COMPARATIVO COMPLETO (H2O + SKLEARN + GPU):")
print("=" * 100)

# Formatear tabla para mejor visualización
display_cols = ['model', 'framework', 'rmse', 'r2', 'training_time']
results_display = results_df[display_cols].copy()

results_display['rmse'] = results_display['rmse'].round(0)
results_display['r2'] = results_display['r2'].round(4)
results_display['training_time'] = results_display['training_time'].round(2)

# Ordenar por R² descendente
results_display = results_display.sort_values('r2', ascending=False)
print(results_display.to_string(index=False))

# Análisis por framework
print(f"\n🏆 ANÁLISIS POR FRAMEWORK:")
print("=" * 40)

framework_analysis = results_df.groupby('framework').agg({
    'rmse': ['mean', 'min'],
    'r2': ['mean', 'max'],
    'training_time': ['mean', 'sum']
}).round(3)

print(framework_analysis)

# Top 3 modelos globales
print(f"\n🥇 TOP 3 MODELOS GLOBALES:")
print("-" * 40)

top_models = results_df.nlargest(3, 'r2')
for i, (_, model) in enumerate(top_models.iterrows(), 1):
    print(f"  {i}. {model['model']} ({model['framework']})")
    print(f"     R² = {model['r2']:.4f}, RMSE = {model['rmse']:,.0f}")
    print(f"     Tiempo = {model['training_time']:.1f}s")

# Comparación de velocidad por framework
h2o_time = results_df[results_df['framework'] == 'H2O']['training_time'].sum()
sklearn_time = results_df[results_df['framework'] == 'Sklearn']['training_time'].sum()

print(f"\n⚡ COMPARACIÓN DE VELOCIDAD:")
print(f"   H2O total: {h2o_time:.1f}s")
print(f"   Sklearn total: {sklearn_time:.1f}s")
if h2o_time > 0 and sklearn_time > 0:
    speedup = sklearn_time / h2o_time
    print(f"   Speedup H2O vs Sklearn: {speedup:.1f}x")

# Identificar mejor modelo overall
best_model_idx = results_df['r2'].idxmax()
best_model_info = results_df.loc[best_model_idx]

print(f"\n🎯 MEJOR MODELO OVERALL: {best_model_info['model']} ({best_model_info['framework']})")
print(f"   R² = {best_model_info['r2']:.4f}")
print(f"   RMSE = {best_model_info['rmse']:,.0f}")
print(f"   Tiempo = {best_model_info['training_time']:.1f}s")

# Guardar resultados completos
model_results['comprehensive_evaluation'] = results_df
model_results['best_overall_model'] = best_model_info['model']
model_results['framework_analysis'] = framework_analysis

print(f"\n✅ Evaluación comparativa completa finalizada")

In [ ]:
# ==================================================
# 4.2 VISUALIZACIÓN COMPARATIVA CON MÉTRICAS DE RENDIMIENTO
# ==================================================

print("📈 Generando visualizaciones comparativas con análisis de rendimiento...")

# 1. Dashboard de métricas principales incluyendo velocidad
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# RMSE comparison
bars1 = axes[0,0].bar(results_df['model'], results_df['rmse'], color='lightcoral', alpha=0.8)
axes[0,0].set_title('RMSE por Modelo', fontsize=14, fontweight='bold')
axes[0,0].set_ylabel('RMSE (DKK)')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(True, alpha=0.3)
# Añadir valores en las barras
for bar in bars1:
    height = bar.get_height()
    axes[0,0].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:,.0f}', ha='center', va='bottom', fontsize=10)

# R² comparison
bars2 = axes[0,1].bar(results_df['model'], results_df['r2'], color='lightblue', alpha=0.8)
axes[0,1].set_title('R² por Modelo', fontsize=14, fontweight='bold')
axes[0,1].set_ylabel('R²')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].grid(True, alpha=0.3)
for bar in bars2:
    height = bar.get_height()
    axes[0,1].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}', ha='center', va='bottom', fontsize=10)

# Training Time comparison
bars3 = axes[0,2].bar(results_df['model'], results_df['training_time'], color='gold', alpha=0.8)
axes[0,2].set_title('Tiempo de Entrenamiento', fontsize=14, fontweight='bold')
axes[0,2].set_ylabel('Tiempo (segundos)')
axes[0,2].tick_params(axis='x', rotation=45)
axes[0,2].grid(True, alpha=0.3)
for bar in bars3:
    height = bar.get_height()
    axes[0,2].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.1f}s', ha='center', va='bottom', fontsize=10)

# MAE comparison
bars4 = axes[1,0].bar(results_df['model'], results_df['mae'], color='lightgreen', alpha=0.8)
axes[1,0].set_title('MAE por Modelo', fontsize=14, fontweight='bold')
axes[1,0].set_ylabel('MAE (DKK)')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(True, alpha=0.3)
for bar in bars4:
    height = bar.get_height()
    axes[1,0].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:,.0f}', ha='center', va='bottom', fontsize=10)

# Efficiency Score (R²/Time)
efficiency_vals = results_df['r2'] / results_df['training_time']
bars5 = axes[1,1].bar(results_df['model'], efficiency_vals, color='purple', alpha=0.8)
axes[1,1].set_title('Eficiencia (R²/Tiempo)', fontsize=14, fontweight='bold')
axes[1,1].set_ylabel('Eficiencia')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(True, alpha=0.3)
for bar in bars5:
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}', ha='center', va='bottom', fontsize=10)

# Scatter: Performance vs Speed
axes[1,2].scatter(results_df['training_time'], results_df['r2'], 
                  s=results_df['rmse']/100, alpha=0.7, c=range(len(results_df)), 
                  cmap='viridis')
for i, row in results_df.iterrows():
    axes[1,2].annotate(row['model'], (row['training_time'], row['r2']), 
                      xytext=(5, 5), textcoords='offset points', fontsize=9)
axes[1,2].set_xlabel('Tiempo de Entrenamiento (s)')
axes[1,2].set_ylabel('R²')
axes[1,2].set_title('Performance vs Velocidad\n(tamaño = RMSE)')
axes[1,2].grid(True, alpha=0.3)

plt.suptitle('Dashboard Comparativo - Precisión y Rendimiento', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.show()

# 2. Análisis de trade-off Performance-Velocidad
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Pareto Front: R² vs Training Time
axes[0].scatter(results_df['training_time'], results_df['r2'], s=100, alpha=0.7)
for i, row in results_df.iterrows():
    axes[0].annotate(row['model'], (row['training_time'], row['r2']), 
                    xytext=(5, 5), textcoords='offset points', fontsize=10)

# Identificar frontera de Pareto
pareto_indices = []
for i, row_i in results_df.iterrows():
    is_pareto = True
    for j, row_j in results_df.iterrows():
        if i != j:
            # Un punto está dominado si otro tiene mejor R² Y menor tiempo
            if row_j['r2'] >= row_i['r2'] and row_j['training_time'] <= row_i['training_time']:
                if row_j['r2'] > row_i['r2'] or row_j['training_time'] < row_i['training_time']:
                    is_pareto = False
                    break
    if is_pareto:
        pareto_indices.append(i)

# Dibujar frontera de Pareto
pareto_points = results_df.iloc[pareto_indices].sort_values('training_time')
axes[0].plot(pareto_points['training_time'], pareto_points['r2'], 'r--', alpha=0.7, linewidth=2, label='Frontera de Pareto')
axes[0].scatter(pareto_points['training_time'], pareto_points['r2'], c='red', s=150, alpha=0.8, marker='*')

axes[0].set_xlabel('Tiempo de Entrenamiento (s)')
axes[0].set_ylabel('R²')
axes[0].set_title('Trade-off Performance vs Velocidad')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Radar chart para comparación multidimensional
categories = ['R²', 'Velocidad\n(1/tiempo)', 'Precisión\n(1/RMSE)', 'Estabilidad\n(1/MAE)']
N = len(categories)

# Normalizar métricas para radar chart
r2_norm = (results_df['r2'] - results_df['r2'].min()) / (results_df['r2'].max() - results_df['r2'].min())
speed_norm = (1/results_df['training_time'] - 1/results_df['training_time'].max()) / (1/results_df['training_time'].min() - 1/results_df['training_time'].max())
rmse_norm = (1/results_df['rmse'] - 1/results_df['rmse'].max()) / (1/results_df['rmse'].min() - 1/results_df['rmse'].max())
mae_norm = (1/results_df['mae'] - 1/results_df['mae'].max()) / (1/results_df['mae'].min() - 1/results_df['mae'].max())

# Ángulos para radar chart
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]

# Plot top 3 modelos en radar chart
top_3_models = ranking_scores[:3]
colors = ['blue', 'red', 'green']

for i, model_info in enumerate(top_3_models):
    model_name = model_info['model']
    model_idx = results_df[results_df['model'] == model_name].index[0]
    
    values = [r2_norm.iloc[model_idx], speed_norm.iloc[model_idx], 
              rmse_norm.iloc[model_idx], mae_norm.iloc[model_idx]]
    values += values[:1]
    
    axes[1].plot(angles, values, 'o-', linewidth=2, label=model_name, color=colors[i])
    axes[1].fill(angles, values, alpha=0.25, color=colors[i])

axes[1].set_xticks(angles[:-1])
axes[1].set_xticklabels(categories)
axes[1].set_ylim(0, 1)
axes[1].set_title('Comparación Multidimensional\n(Top 3 Modelos)')
axes[1].legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
axes[1].grid(True)

plt.tight_layout()
plt.show()

# 3. Resumen de optimizaciones aplicadas
print(f"\n📊 RESUMEN DE OPTIMIZACIONES APLICADAS:")
print("=" * 60)

optimization_summary = {
    'GPU_LightGBM': 'Sí' if GPU_AVAILABLE else 'No disponible',
    'Paralelización_RF': f'{mp.cpu_count()} cores',
    'GridSearch_Paralelo': 'Todos los modelos',
    'Memoria_Optimizada': f'{parallel_config["available_memory_gb"]} GB disponible',
    'Tiempo_Total': f'{sum([m["training_time"] for m in evaluation_results]):.1f}s'
}

for key, value in optimization_summary.items():
    print(f"  {key.replace('_', ' ')}: {value}")

# Comparación de speedup teórico vs real
theoretical_speedup = mp.cpu_count()
actual_times = {m['model']: m['training_time'] for m in evaluation_results}

print(f"\n⚡ ANÁLISIS DE SPEEDUP:")
print("-" * 30)
print(f"Speedup teórico máximo: {theoretical_speedup}x")
print(f"Modelo más rápido: {min(actual_times, key=actual_times.get)} ({min(actual_times.values()):.1f}s)")
print(f"Modelo más lento: {max(actual_times, key=actual_times.get)} ({max(actual_times.values()):.1f}s)")
print(f"Speedup real LightGBM vs RF: {actual_times.get('Random_Forest', 1)/actual_times.get('LightGBM', 1):.1f}x")

print(f"\n✅ Visualización optimizada completada con métricas de rendimiento")

# CARGA DE DATOS PROCESADOS DESDE FEATURE ENGINEERING

print("📂 Cargando datos procesados desde notebook 03 (ingeniería de características)...")

# Usar configuración para rutas de datos procesados
processed_data_path = DATA_DIR / "processed"

# Buscar archivos de datos procesados específicos
possible_files = [
    "feature_engineered_complete.parquet",
    "modeling_dataset.parquet", 
    "cleaned_data.parquet",
    "processed_data.parquet"
]

df_processed = None
loaded_file = None

# Intentar cargar archivos en orden de prioridad
for filename in possible_files:
    file_path = processed_data_path / filename
    if file_path.exists():
        try:
            if filename.endswith('.parquet'):
                df_processed = pd.read_parquet(file_path)
            else:
                df_processed = pd.read_csv(file_path)
            
            loaded_file = filename
            print(f"✅ Cargado exitosamente: {filename}")
            break
        except Exception as e:
            print(f"⚠️ Error cargando {filename}: {e}")

# Fallback a datos limpios si no se encuentra data procesada
if df_processed is None:
    print("⚠️ No se encontraron datos procesados, usando datos limpios como fallback")
    df_processed = df_clean.copy()
    loaded_file = "cleaned_data (fallback)"

print(f"📊 Datos cargados desde: {loaded_file}")
print(f"   Dimensiones: {df_processed.shape[0]:,} filas x {df_processed.shape[1]} columnas")

# Verificar integridad de datos procesados
print(f"\n🔍 Verificando datos procesados:")
print(f"   Target '{TARGET}' presente: {TARGET in df_processed.columns}")
print(f"   Valores nulos totales: {df_processed.isnull().sum().sum():,}")

# Limpiar datos si es necesario
if TARGET in df_processed.columns:
    initial_rows = len(df_processed)
    df_processed = df_processed.dropna(subset=[TARGET])
    if initial_rows != len(df_processed):
        print(f"   Limpieza: {initial_rows} → {len(df_processed)} filas")

# Actualizar configuraciones para datos procesados
categorical_cols = df_processed.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()

# Preparar features para modelado (excluir target y columnas problemáticas)
available_features = [col for col in df_processed.columns 
                     if col not in (DROP_COLS + [TARGET])]

print(f"   Features disponibles: {len(available_features)}")
print(f"   Categóricas: {len(categorical_cols)}, Numéricas: {len(numerical_cols)}")

print(f"\n✅ Preparación de datos procesados completada")

In [ ]:
# ==================================================
# CONVERSIÓN A H2O FRAME Y PREPARACIÓN MULTIFRAMEWORK
# ==================================================

print("🌊 Preparando datos para frameworks múltiples (H2O + Sklearn)...")

# Convertir a H2O Frame si H2O está disponible
if H2O_AVAILABLE:
    print("🔄 Convirtiendo datos procesados a H2O Frame...")
    
    try:
        # Crear copia para H2O (evitar modificar original)
        df_h2o_prep = df_processed.copy()
        
        # Limpiar datos para H2O (remover filas con nulls en target)
        if TARGET in df_h2o_prep.columns:
            initial_rows = len(df_h2o_prep)
            df_h2o_prep = df_h2o_prep.dropna(subset=[TARGET])
            cleaned_rows = len(df_h2o_prep)
            if initial_rows != cleaned_rows:
                print(f"   Limpieza H2O: {initial_rows} → {cleaned_rows} filas (-{initial_rows-cleaned_rows})")
        
        # Convertir DataFrame a H2O Frame
        h2o_frame = h2o.H2OFrame(df_h2o_prep)
        h2o_frame.set_names(list(df_h2o_prep.columns))
        
        # Configurar tipos de datos en H2O
        if TARGET in df_h2o_prep.columns:
            h2o_frame[TARGET] = h2o_frame[TARGET].asnumeric()
        
        # Convertir columnas categóricas en H2O
        for col in categorical_cols:
            if col in h2o_frame.columns:
                try:
                    h2o_frame[col] = h2o_frame[col].asfactor()
                except:
                    print(f"   ⚠️ No se pudo convertir {col} a factor")
        
        # Convertir columnas numéricas explícitamente
        for col in numerical_cols:
            if col in h2o_frame.columns and col != TARGET:
                try:
                    h2o_frame[col] = h2o_frame[col].asnumeric()
                except:
                    print(f"   ⚠️ No se pudo convertir {col} a numérico")
        
        print(f"✅ H2O Frame creado exitosamente:")
        print(f"   Dimensiones: {h2o_frame.nrows:,} filas x {h2o_frame.ncols} columnas")
        print(f"   Memoria H2O Frame: {h2o_frame.byte_size / (1024**2):.1f} MB")
        
        # Mostrar información del frame H2O
        if TARGET in df_h2o_prep.columns:
            print(f"\n📊 Información H2O Frame - Variable objetivo:")
            target_summary = h2o_frame[TARGET].summary()
            target_summary.show()
        
        # Verificar tipos en H2O
        print(f"\n🔍 Tipos de datos en H2O:")
        h2o_types = h2o_frame.types
        type_counts = {}
        for col, dtype in h2o_types.items():
            type_counts[dtype] = type_counts.get(dtype, 0) + 1
        
        for dtype, count in type_counts.items():
            print(f"   {dtype}: {count} columnas")
        
    except Exception as e:
        print(f"❌ Error creando H2O Frame: {e}")
        h2o_frame = None
        H2O_AVAILABLE = False
        
else:
    print("⚠️ H2O no disponible, trabajando solo con pandas/sklearn")
    h2o_frame = None

# Preparar datos para sklearn (siempre disponible como backup)
print(f"\n📊 Preparando datos para Sklearn...")

# Verificar y limpiar datos para sklearn
df_sklearn = df_processed.copy()

# Remover filas con nulls en target
if TARGET in df_sklearn.columns:
    initial_rows = len(df_sklearn)
    df_sklearn = df_sklearn.dropna(subset=[TARGET])
    if initial_rows != len(df_sklearn):
        print(f"   Limpieza Sklearn: {initial_rows} → {len(df_sklearn)} filas")

# Filtrar features disponibles para sklearn
available_features_sklearn = [f for f in available_features if f in df_sklearn.columns]

# Verificar que tenemos datos válidos
if not available_features_sklearn:
    print("❌ Error: No hay features válidas para sklearn")
    print(f"   Features solicitadas: {available_features[:10]}...")
    print(f"   Columnas disponibles: {list(df_sklearn.columns)}")
    raise ValueError("No hay features válidas para modelado")

if TARGET not in df_sklearn.columns:
    print(f"❌ Error: Variable objetivo '{TARGET}' no está en los datos")
    raise ValueError(f"Variable objetivo '{TARGET}' no encontrada")

print(f"✅ Datos Sklearn preparados:")
print(f"   Dimensiones: {len(df_sklearn):,} filas x {len(available_features_sklearn)} features")
print(f"   Variable objetivo: {TARGET}")

# Configuración final para modelado
modeling_config = {
    'target_column': TARGET,
    'feature_columns': available_features_sklearn,
    'categorical_columns': [col for col in categorical_cols if col in available_features_sklearn],
    'numerical_columns': [col for col in numerical_cols if col in available_features_sklearn],
    'h2o_available': H2O_AVAILABLE and h2o_frame is not None,
    'total_samples': len(df_sklearn),
    'total_features': len(available_features_sklearn)
}

print(f"\n🎯 Configuración final de modelado:")
for key, value in modeling_config.items():
    if isinstance(value, list) and len(value) > 5:
        print(f"   {key}: {len(value)} elementos")
    else:
        print(f"   {key}: {value}")

print(f"\n✅ Preparación multiframework completada")
print(f"🚀 Listo para modelado con H2O: {modeling_config['h2o_available']}")

🌊 Preparando datos para frameworks múltiples (H2O + Sklearn)...
⚠️ H2O no disponible, trabajando solo con pandas/sklearn

📊 Preparando datos para Sklearn...
✅ Datos Sklearn preparados:
   Dimensiones: 1,506,591 filas x 14 features
   Variable objetivo: purchase_price

🎯 Configuración final de modelado:
   target_column: purchase_price
   feature_columns: 14 elementos
   categorical_columns: 7 elementos
   numerical_columns: 7 elementos
   h2o_available: False
   total_samples: 1506591
   total_features: 14

✅ Preparación multiframework completada
🚀 Listo para modelado con H2O: False


In [ ]:
# ==================================================
# DIVISIÓN H2O FRAME Y PREPARACIÓN DISTRIBUIDA
# ==================================================

if H2O_AVAILABLE and h2o_frame is not None:
    print("🌊 Preparando división H2O con datos procesados...")
    
    try:
        # Configurar división H2O usando las mismas features
        h2o_features = [f for f in final_features if f in h2o_frame.columns]
        
        print(f"   Features H2O: {len(h2o_features)}")
        print(f"   Target H2O: {TARGET}")
        
        # Verificar que el target esté disponible
        if TARGET not in h2o_frame.columns:
            print(f"❌ Target '{TARGET}' no encontrado en H2O Frame")
            print(f"   Columnas disponibles: {h2o_frame.columns}")
            raise ValueError(f"Target no encontrado en H2O")
        
        # División train/validation/test para H2O (80/10/10)
        # Mantener consistencia con división sklearn cuando sea posible
        train_ratio = 0.8
        valid_ratio = 0.1
        test_ratio = 0.1
        
        # División temporal si hay columnas de fecha en H2O
        temporal_h2o = False
        if temporal_columns:
            try:
                date_col = temporal_columns[0]
                if date_col in h2o_frame.columns:
                    print(f"   Usando división temporal H2O con {date_col}")
                    
                    # Ordenar por fecha para H2O
                    h2o_frame_sorted = h2o_frame.sort(date_col)
                    
                    # Calcular puntos de corte
                    total_rows = h2o_frame_sorted.nrows
                    train_end = int(total_rows * train_ratio)
                    valid_end = int(total_rows * (train_ratio + valid_ratio))
                    
                    # Crear splits temporales
                    train_h2o = h2o_frame_sorted[:train_end, :]
                    valid_h2o = h2o_frame_sorted[train_end:valid_end, :]
                    test_h2o = h2o_frame_sorted[valid_end:, :]
                    
                    temporal_h2o = True
                    print(f"   ✅ División temporal H2O exitosa")
                    print(f"   Train H2O: {train_h2o.nrows:,} filas")
                    print(f"   Valid H2O: {valid_h2o.nrows:,} filas") 
                    print(f"   Test H2O: {test_h2o.nrows:,} filas")
                    
            except Exception as e:
                print(f"   ⚠️ Error división temporal H2O: {e}")
                temporal_h2o = False
        
        # División aleatoria como fallback para H2O
        if not temporal_h2o:
            print(f"   Usando división aleatoria H2O")
            
            # División aleatoria balanceada
            train_h2o, valid_h2o, test_h2o = h2o_frame.split_frame(
                ratios=[train_ratio, valid_ratio], 
                seed=42
            )
            
            print(f"   ✅ División aleatoria H2O exitosa")
            print(f"   Train H2O: {train_h2o.nrows:,} filas")
            print(f"   Valid H2O: {valid_h2o.nrows:,} filas")
            print(f"   Test H2O: {test_h2o.nrows:,} filas")
        
        # Configurar predictores y target para H2O
        h2o_predictors = h2o_features
        h2o_target = TARGET
        
        # Validar que tenemos datos suficientes
        min_samples = 100
        if train_h2o.nrows < min_samples:
            print(f"⚠️ Advertencia: Train H2O tiene solo {train_h2o.nrows} muestras")
        
        # Información de la división H2O
        print(f"\n📊 Configuración H2O final:")
        print(f"   Predictores: {len(h2o_predictors)}")
        print(f"   Target: {h2o_target}")
        print(f"   División temporal: {temporal_h2o}")
        
        # Verificar tipos de datos en cada split
        print(f"\n? Verificación de tipos H2O:")
        target_type_train = train_h2o[h2o_target].type
        print(f"   Target type: {target_type_train}")
        
        # Estadísticas del target en H2O
        print(f"\n📈 Estadísticas target H2O:")
        train_target_summary = train_h2o[h2o_target].summary()
        print(f"   Train target summary disponible")
        
        h2o_ready = True
        print(f"\n✅ H2O Frame división completada")
        
    except Exception as e:
        print(f"❌ Error en división H2O: {e}")
        h2o_ready = False
        train_h2o = None
        valid_h2o = None
        test_h2o = None
        h2o_predictors = []
        h2o_target = None
        
else:
    print("⚠️ H2O no disponible para división")
    h2o_ready = False
    train_h2o = None
    valid_h2o = None
    test_h2o = None
    h2o_predictors = []
    h2o_target = None


In [ ]:

# Resumen de división multiframework
print(f"\n🎯 Resumen división multiframework:")
print(f"   Sklearn listo: ✅ ({len(X_train):,} train, {len(X_test):,} test)")
print(f"   H2O listo: {'✅' if h2o_ready else '❌'} {f'({train_h2o.nrows:,} train)' if h2o_ready else ''}")
print(f"   Features finales: {len(final_features)}")

# Guardar configuración para modelos
model_config = {
    'sklearn_ready': True,
    'h2o_ready': h2o_ready,
    'features': final_features,
    'target': TARGET,
    'train_size': len(X_train),
    'test_size': len(X_test),
    'h2o_train_size': train_h2o.nrows if h2o_ready else 0
}

print(f"\n🚀 Configuración almacenada para modelado")

In [ ]:
# ==================================================
# 4.3 RESUMEN FINAL Y PERSISTENCIA DE RESULTADOS
# ==================================================

print("📋 Generando resumen final del modelado supervisado...")

# Resumen estadístico final
print("\n" + "="*60)
print("📊 RESUMEN EJECUTIVO - MODELADO SUPERVISADO")
print("="*60)

print(f"\n🎯 CONFIGURACIÓN DEL EXPERIMENTO:")
print(f"  • Dataset: Precios inmobiliarios Dinamarca")
print(f"  • Tamaño total: {len(df_processed):,} observaciones")
print(f"  • Training set: {len(X_train):,} observaciones ({len(X_train)/len(df_processed)*100:.1f}%)")
print(f"  • Test set: {len(X_test):,} observaciones ({len(X_test)/len(df_processed)*100:.1f}%)")
print(f"  • Features utilizadas: {len(available_features)} variables")
print(f"  • Período temporal: {df_processed['Year'].min()}-{df_processed['Year'].max()}")

print(f"\n🏆 RESULTADOS PRINCIPALES:")
best_model_metrics = results_df[results_df['model'] == best_model_name].iloc[0]
print(f"  • Mejor modelo: {best_model_name}")
print(f"  • RMSE: {best_model_metrics['rmse']:,.0f} DKK")
print(f"  • MAE: {best_model_metrics['mae']:,.0f} DKK")
print(f"  • R²: {best_model_metrics['r2']:.4f}")
print(f"  • MAPE: {best_model_metrics['mape']:.2f}%")

# Contexto de precios para interpretar errores
price_stats = y_test.describe()
print(f"\n💰 CONTEXTO DE PRECIOS (conjunto de test):")
print(f"  • Precio promedio: {price_stats['mean']:,.0f} DKK")
print(f"  • Precio mediano: {price_stats['50%']:,.0f} DKK")
print(f"  • Rango de precios: {price_stats['min']:,.0f} - {price_stats['max']:,.0f} DKK")
print(f"  • Error relativo RMSE: {best_model_metrics['rmse_relative']:.1f}% del rango total")

print(f"\n📈 MEJORAS RESPECTO A BASELINE:")
baseline_metrics = results_df[results_df['model'] == 'Linear_Regression'].iloc[0]
improvement_rmse = ((baseline_metrics['rmse'] - best_model_metrics['rmse']) / baseline_metrics['rmse']) * 100
improvement_r2 = ((best_model_metrics['r2'] - baseline_metrics['r2']) / baseline_metrics['r2']) * 100

print(f"  • Reducción RMSE: {improvement_rmse:.1f}%")
print(f"  • Mejora R²: {improvement_r2:.1f}%")
print(f"  • Ahorro en error promedio: {baseline_metrics['mae'] - best_model_metrics['mae']:,.0f} DKK")

# Top features del mejor modelo
if best_model_name == 'LightGBM':
    best_feature_imp = model_results['lgb_feature_importance'].head(5)
elif best_model_name == 'Random_Forest':
    best_feature_imp = model_results['rf_feature_importance'].head(5)

if best_model_name in ['LightGBM', 'Random_Forest']:
    print(f"\n🎯 TOP 5 FEATURES MÁS IMPORTANTES ({best_model_name}):")
    for i, (_, row) in enumerate(best_feature_imp.iterrows(), 1):
        print(f"  {i}. {row['feature']}")

# Estadísticas del proceso de modelado
print(f"\n⚙️ ESTADÍSTICAS DEL PROCESO:")
print(f"  • Modelos evaluados: {len(results_df)} algoritmos")
print(f"  • Tiempo de división temporal validado: ✓")
print(f"  • Análisis de residuos completado: ✓")
print(f"  • Feature importance analizado: ✓")
print(f"  • Criterios de evaluación: 4 métricas principales")

# Guardar resumen final
final_summary = {
    'experiment_config': {
        'total_samples': len(df_processed),
        'train_samples': len(X_train),
        'test_samples': len(X_test),
        'n_features': len(available_features),
        'time_period': f"{df_processed['Year'].min()}-{df_processed['Year'].max()}"
    },
    'best_model': {
        'name': best_model_name,
        'metrics': best_model_metrics.to_dict()
    },
    'price_context': price_stats.to_dict(),
    'improvements': {
        'rmse_improvement_pct': improvement_rmse,
        'r2_improvement_pct': improvement_r2,
        'mae_savings_dkk': baseline_metrics['mae'] - best_model_metrics['mae']
    },
    'all_models_summary': results_df.to_dict('records')
}

model_results['final_summary'] = final_summary

# Preparar datos para exportación
export_data = {
    'model_performance': results_df,
    'best_model_name': best_model_name,
    'feature_names': available_features,
    'train_test_split': {
        'train_size': len(X_train),
        'test_size': len(X_test),
        'split_date': None  # Se podría añadir fecha de corte si aplica
    }
}

print(f"\n💾 DATOS PREPARADOS PARA EXPORTACIÓN:")
print(f"  • Métricas de rendimiento: ✓")
print(f"  • Modelo recomendado: {best_model_name}")
print(f"  • Features importantes: ✓")
print(f"  • Configuración train/test: ✓")

print(f"\n✅ Modelado supervisado completado exitosamente")
print(f"🎯 Recomendación: Proceder con {best_model_name} para implementación en producción")

print("\n" + "="*60)
print("FIN DEL ANÁLISIS DE MODELADO SUPERVISADO")
print("="*60)

## 4. Comparación de métricas

## 5. Interpretación con SHAP y/o LIME